# Imports/Pick Analyses to run

In [1]:
import warnings

import mne
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.animation import FuncAnimation
import matplotlib.patches as mpatches
import os
import json
import numpy as np
import itertools
import pandas as pd
from collections import defaultdict
from collections import Counter
import re
import seaborn as sns
from statsmodels.formula.api import mixedlm, ols
from scipy import stats
from statsmodels.stats.multitest import fdrcorrection
import statsmodels.api as sm

from preprocess_nirs import *
from nirs_functions import *

from mne_nirs.channels import picks_pair_to_idx, get_long_channels
from mne_nirs.preprocessing import peak_power, scalp_coupling_index_windowed
from mne.preprocessing.nirs import source_detector_distances, scalp_coupling_index
from mne_connectivity import spectral_connectivity_epochs, spectral_connectivity_time
from mne_connectivity.viz import plot_connectivity_circle
from mne.viz import circular_layout
from mne_nirs.experimental_design import make_first_level_design_matrix
from mne_nirs.statistics import run_glm, statsmodels_to_results

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score, GridSearchCV, GroupKFold
from sklearn.pipeline import make_pipeline

import tensorflow as tf

# Set the image dpi
dpi = 600

# Set the number of cores to use
n_jobs = 1
if os.cpu_count() is not None:
    n_jobs = int(np.ceil(os.cpu_count() * 0.75))

seed_value = 42
# Set the random seed
os.environ['PYTHONHASHSEED'] = str(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.keras.utils.set_random_seed(seed_value)
tf.config.experimental.enable_op_determinism()

# Get the mappings of the brain regions
mappings = json.load(open("processed_data/mappings/mappings.json"))

# Set thresholds for the data
sci_threshold = 0.5
cv_threshold = 15
good_threshold = 0.7
peak_power_threshold = 0.1
behavioural_accuracy_threshold = 0.6

# Preprocess the data
process_data = False

# Get the participant info plots
get_participant_age_plot = False
get_meas_dates_plot = False
get_short_chns_plot = False
get_behavioural_responses = False
run_behavioural_responses_stats = False
get_behavioural_responses_plots = False

# Get the mappings
get_mappings = False

# Get the non windowed SCI and CV measure
get_full_sci = False
get_full_cv = False

# Get the peak power/scalp coupling index dataframes
get_peak_power_sci_df = False
get_good_windows_plot = True
get_slope_pp_sci_plot = False
get_head_size_vs_sci_plot = False
get_across_participant_sci_plots = False
get_participant_sci_plots = False

# Get the epochs
get_epochs = False
get_face_type_emotion_epochs = False

# Get the GLM data/plots
get_glm_analysis = False
get_ind_glm_plots = False
get_group_glm_plots = False
get_group_contrast_plots = False

# Get the average timeseries activity
get_roi_timeseries_activity = False

# Get the ERP plots
get_erp_plots = False

# Get the connectivity data
run_ind_connectivity = False
get_condition_con_plots = False
get_condition_con_gif_plots = False
get_variance_con_plots = False
run_group_level_t_tests = False
get_group_level_t_tests_chord_plots = False
get_group_level_t_tests_roi_chord_plots = False

# Run decoding analysis
run_traditional_raw_within_decoding = False
run_traditional_con_within_decoding = False
run_traditional_raw_across_decoding = True
run_dl_raw_across_decoding = False
run_traditional_con_across_decoding = False
run_dl_con_across_decoding = False

# Get the decoding plots
get_decoding_table_scores_plots = False
get_decoding_individual_scores_plots = False

raws = []
raw_ods = []
raw_haemos = []
raw_haemo_good_recordings = []

# Get Participants

In [2]:
# Define the folder path
data_path = os.path.join(os.getcwd(), 'data')

# Get a list of paths of all the subfolders of the folders labeled 'P_1', 'P_2', etc.
participants = [os.path.join(data_path, f) for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]

participants_with_same_order = []

# remove participants P_1 to P_11 but keep P_10, and P_12 onwards
# P_1 to P_11 have the same order of faces, and P_86 and P_87 have the same order of faces
for i in range(1, 12):
    if i != 10:
        participants_with_same_order.append(os.path.join(data_path, f'P_{i}'))
        #participants.remove(os.path.join(data_path, f'P_{i}'))

participants_with_same_order.append(os.path.join(data_path, f'P_87'))
#participants.remove(os.path.join(data_path, f'P_87'))

# remove participants P_13 due to not recording
participants.remove(os.path.join(data_path, f'P_13'))

# remove participants P_50 due to ending early
participants.remove(os.path.join(data_path, f'P_50'))

# participant P_54 used their phone
participants.remove(os.path.join(data_path, f'P_54'))

# Search recursively for the folder with the .snirf extension
fnirs_folders = []
for participant in participants:
    for root, dirs, files in os.walk(participant):
        for file in files:
            if file.endswith('.snirf'):
                fnirs_folders.append(root)
                break

# Preprocessing

In [3]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    if process_data:
        raws = []
        raw_ods = []
        raw_haemos = []

        # Load the snirf files
        for folder in fnirs_folders:
            # find all the .snirf files in the folder but get the full path
            snirf_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.snirf')]
            if len(snirf_files) == 0:
                print(f"No .snirf files found in {folder}")
                continue
            elif len(snirf_files) > 1:
                raise Exception(f"Multiple .snirf files found in {folder}")
            else:
                raw = mne.io.read_raw_snirf(snirf_files[0], optode_frame='mri', preload=True, verbose=False)
            
            # find all the 'description.json' files in the folder but get the full path
            description_files = [f for f in os.listdir(folder) if 'description.json' in f]
            if len(description_files) == 0:
                print(f"No description.json files found in {folder}")
                continue
            elif len(description_files) > 1:
                raise Exception(f"Multiple description.json files found in {folder}")
            else:
                description = json.load(open(os.path.join(folder, description_files[0])))
                # get the parent's parent folder name
                parent_folder = os.path.basename(os.path.dirname(folder))
                # find the .csv file in the parent folder
                csv_files = [f for f in os.listdir(os.path.join(data_path, parent_folder)) if f.endswith('.csv')]
                if len(csv_files) == 0:
                    print(f"No .csv files found in {os.path.join(data_path, parent_folder)}")
                elif len(csv_files) > 1:
                    raise Exception(f"Multiple .csv files found in {os.path.join(data_path, parent_folder)}")
                else:
                    # add the path to the .csv file to the description
                    description['behavioural_data'] = os.path.join(data_path, parent_folder, csv_files[0])

            distances = np.array(source_detector_distances(raw.info))
            distance_counts = {
                "short": int(sum(distances < 0.01)),
                "long": int(sum(distances >= 0.01))
            }

            description['distance_counts'] = distance_counts

            # add the description to the raw object
            raw.info['description'] = str(description)
            raws.append(raw)

        # sort the raws by the measurement date
        raws = sorted(raws, key=lambda x: x.info['meas_date'])

        i = 1
        for raw in raws:
            raw_od, raw_haemo = preprocess_data(raw)
            raw_ods.append(raw_od)
            raw_haemos.append(raw_haemo)
            i += 1

        # clear any files in each folder
        for folder in ['processed_data/raws', 'processed_data/raw_ods', 'processed_data/raw_haemos']:
            for f in os.listdir(folder):
                os.remove(os.path.join(folder, f))

        for i, (raw, raw_od, raw_haemo) in enumerate(zip(raws, raw_ods, raw_haemos), 1):
            # save raw as a fif file
            raw.save(f'processed_data/raws/raw{i}.fif', overwrite=True, verbose=False)

            # save raw_od as a fif file
            raw_od.save(f'processed_data/raw_ods/raw_od{i}.fif', overwrite=True, verbose=False)

            # save raw_haemo as a fif file
            raw_haemo.save(f'processed_data/raw_haemos/raw_haemo{i}.fif', overwrite=True, verbose=False)
        
        raws = []
        raw_ods = []
        raw_haemos = []

# Participant Information

### Participant Age

In [14]:
if get_participant_age_plot:
    raw_haemo_good_recordings = load_data('raw_haemos', 'good') if len(raw_haemo_good_recordings) == 0 else raw_haemo_good_recordings

    ages = []
    for raw_haemo in raw_haemo_good_recordings:
        description = get_info(raw_haemo)
        age = description['age']
        ages.append(age)

    # Convert ages to numeric type
    ages = np.array(ages, dtype=float)

    # Sort the ages
    ages = sorted(ages)

    # Get the number of participants
    n_participants = len(ages)

    # Get the mean and standard deviation of the ages
    mean_age = np.mean(ages)
    std_age = np.std(ages)

    # Get the minimum and maximum ages
    min_age = min(ages)
    max_age = max(ages)
    
    # Plot the ages in a histogram
    plt.figure(figsize=(8, 6))
    plt.hist(ages, bins=30, color='blue', alpha=0.7)
    plt.xlabel('Age (years)', fontsize=14)
    plt.ylabel('Count', fontsize=14)
    plt.title(f'Age Distribution of Participants\nN ={n_participants}, Mean={mean_age:.2f}, SD={std_age:.2f}, Min={min_age}, Max={max_age}', fontsize=16)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(axis='y', alpha=0.75)
    plt.tight_layout()
    plt.savefig('plots/participants/age_distribution.png', dpi=dpi / 4)
    plt.close()

### Measurement Dates

In [ ]:
if get_meas_dates_plot:
    raw_haemos = load_data('raw_haemos') if len(raw_haemos) == 0 else raw_haemos

    # Extract the measurement dates
    measurement_dates = [raw_haemo.info['meas_date'] for raw_haemo in raw_haemos]

    # Convert to pandas datetime
    measurement_dates = pd.to_datetime(measurement_dates)

    # Create a plot of the measurement dates
    plt.figure(figsize=(15, 10))
    plt.plot(measurement_dates, range(1, len(measurement_dates) + 1), 'o-')
    plt.xlabel('Measurement date')
    plt.ylabel('Participant number')
    plt.title('Measurement dates of participants, N = ' + str(len(measurement_dates)))
    plt.grid()
    plt.tight_layout()
    plt.savefig('plots/participants/measurement_dates.png', dpi=dpi / 4)
    plt.close()

### Short Distance Channels Distribution Plot

In [12]:
if get_short_chns_plot:
    raw_haemo_good_recordings = load_data('raw_haemos', 'good') if len(raw_haemo_good_recordings) == 0 else raw_haemo_good_recordings
    
    total_short_chns = []
    for raw_haemo in raw_haemo_good_recordings:
        num_short_chns = get_info(raw_haemo)['distance_counts']['short']
        total_short_chns.append(num_short_chns)
    total_short_chns = np.array(total_short_chns)

    # plot a pie chart of the number of short channels
    plt.figure(figsize=(7, 6))
    plt.pie(Counter(total_short_chns).values(), labels=Counter(total_short_chns).keys(), autopct='%1.1f%%')
    plt.title('Distribution of number of short channels across good recordings, N = ' + str(len(total_short_chns)))
    plt.tight_layout()
    plt.savefig('plots/participants/short_channels.png', dpi=dpi / 4)
    plt.close()

### Generate Behavioural Responses Dataframe

In [ ]:
if get_behavioural_responses:
    raw_haemos = load_data('raw_haemos') if len(raw_haemos) == 0 else raw_haemos

    participant_scores = pd.DataFrame()
    for i, raw_haemo in enumerate(raw_haemos, 1):
        path = get_info(raw_haemo)['behavioural_data']

        # folder_name = (P_(\d+))
        folder_name = re.search(r'P_(\d+)', path).group(0)

        # read the csv file
        behavioural_data = pd.read_csv(path)

        # only keep the first 6 columns and the column called "key_resp.keys"
        behavioural_data = behavioural_data.iloc[:, :6].join(behavioural_data[['key_resp.keys', 'key_resp.rt']])

        # remove the "blocknumber" column
        behavioural_data = behavioural_data.drop(columns='blocknumber')

        # remove the first row and reset the index
        behavioural_data = behavioural_data.iloc[1:].reset_index(drop=True)

        # divide the dataframe into rows of 9
        behavioural_data = [behavioural_data.iloc[i:i + 9] for i in range(0, len(behavioural_data), 9)]

        scores = []
        for j, data in enumerate(behavioural_data, 1):
            # in the last row, save the "key_resp.keys" column
            response = data['key_resp.keys'].iloc[-1]

            # convert the response to a boolean
            response = {'n': False, 'y': True}.get(response, np.nan)
            
            # in the last row, copy the "tasknames" and "taskmarkers" value to "filepaths" and "marker" respectively
            data.loc[data.index[-1], ["filepaths", "marker"]] = [data["tasknames"].iloc[-1], data["taskmarker"].iloc[-1]]

            # remove the "tasknames" and "taskmarker" columns
            data = data.drop(columns=["tasknames", "taskmarker"])

            # check if the (marker - 2000) in the last row was in any of the other rows
            correct_response = (data['marker'].iloc[-1] - 2000) in data['marker'].values

            # append the data to the scores list
            scores.append({
                'Participant': i,
                'Trial': j,
                'Block Name': data['blocknames'].iloc[-1],
                'File Path': data['filepaths'].iloc[-1],
                'Marker': data['marker'].iloc[-1],
                'Response Time': data['key_resp.rt'].iloc[-1],
                'Response': response,
                'Correct Response': correct_response
            })
        
        # append the scores to the participant_scores dataframe
        participant_scores = pd.concat([participant_scores, pd.DataFrame(scores)])

    # for each row, if "Response" == "Correct Response", create a new column called "Correct" and set it to True, else False, if the "Response" is NaN, set it to NaN, don't get rid of the "Correct Response" column
    participant_scores['Correct'] = participant_scores.apply(
        lambda x: np.nan if pd.isna(x['Response']) else bool(x['Response']) == bool(x['Correct Response']), axis=1
    )

    # subtract 2000 from the "Marker" column
    participant_scores['Marker'] = participant_scores['Marker'] - 2000

    # for each row in 'Marker' replace it with two columns 'Face Type' and 'Emotion' from the trigger_decoder function, which returns a tuple
    participant_scores[['Face Type', 'Emotion']] = participant_scores['Marker'].apply(trigger_decoder).apply(pd.Series)

    # remove the "Block Name", "File Path", "Marker", columns
    participant_scores = participant_scores.drop(columns=["Block Name", "File Path", "Marker"])

    # move "Face Type" and "Emotion" to after Participant
    participant_scores = participant_scores[['Participant', 'Trial', 'Face Type', 'Emotion', 'Response Time', 'Response', 'Correct Response', 'Correct']]

    # save the participant_scores dataframe
    participant_scores.to_csv('processed_data/behavioural_responses/participant_scores.csv', index=False)

# Load the participant scores
participant_scores = pd.read_csv('processed_data/behavioural_responses/participant_scores.csv')

# get the number of Correct responses per participant
correct_responses = participant_scores.groupby('Participant')['Correct'].sum()

# get the indices of the participants with > 60% correct responses
good_recordings = correct_responses[correct_responses > 56 * behavioural_accuracy_threshold].index

# only keep the participants with > accuracy_threshold correct responses
participant_scores = participant_scores[participant_scores['Participant'].isin(good_recordings)]

#### Behavioural Responses Statistics

In [ ]:
if run_behavioural_responses_stats:
    # Create a copy of the 'Correct' column and convert it to 1 if True, 0 if False
    participant_scores['Correct Numeric'] = np.where(participant_scores['Correct'] == True, 1, 0)

    # Create a copy of the relevant columns
    data_copy = participant_scores[['Face Type', 'Emotion', 'Correct Numeric']].copy()

    # Rename the column for consistency
    data_copy = data_copy.rename(columns={'Face Type': 'Face_Type', 'Correct Numeric': 'Correct_Numeric'})

    # Build an OLS model including Face Type, Emotion, and their interaction
    model = ols("Correct_Numeric ~ C(Face_Type) * C(Emotion)", data=data_copy).fit()

    # Perform two-way ANOVA
    anova_table = sm.stats.anova_lm(model, typ=3)  # Type 3 ANOVA

    # Print the ANOVA table with p-values not in scientific notation
    anova_table['PR(>F)'] = anova_table['PR(>F)'].apply(lambda x: f"{x:.6f}" if not pd.isnull(x) else x)

    # save the anova table
    anova_table.to_csv('processed_data/behavioural_responses/anova_table.csv')

    p_value_face_type = anova_table.loc['C(Face_Type)', 'PR(>F)']
    p_value_emotion = anova_table.loc['C(Emotion)', 'PR(>F)']
    p_value_interaction = anova_table.loc['C(Face_Type):C(Emotion)', 'PR(>F)']

    # Check if there is a significant correlation between Response Time and Correct
    # Ensure both columns have the same length by dropping NaN values in both simultaneously
    valid_data = participant_scores[['Response Time', 'Correct Numeric']].dropna()
    correlation, p_value = stats.pearsonr(valid_data['Response Time'], valid_data['Correct Numeric'])

#### Behavioural Responses Plots

In [ ]:
if get_behavioural_responses_plots:
    # get the sum of correct responses for each participant
    participant_correct_responses = participant_scores.groupby('Participant')['Correct'].sum()

    n_blocks = len(participant_scores) / len(participant_scores['Participant'].unique())

    # plot this data
    plt.figure(figsize=(10, 6))
    plt.bar(participant_correct_responses.index, participant_correct_responses.values, width=0.9)
    plt.xlabel('Participant')
    plt.ylabel('Number of Correct Responses')
    plt.title('Number of Correct Responses per Participant, Mean across all ' + str(len(participant_correct_responses)) + ' participants: ' + str(round(participant_correct_responses.mean() / n_blocks * 100, 2)) + '%')
    plt.grid()
    plt.tight_layout()
    plt.savefig('plots/behavioural_responses/correct_responses_by_participant.png', dpi=dpi / 4)
    plt.close()

    # get the percentage of correct responses for each type of face and emotion and the combination of both
    face_type_correct_responses = participant_scores.groupby('Face Type')['Correct'].mean() * 100
    emotion_correct_responses = participant_scores.groupby('Emotion')['Correct'].mean() * 100
    face_type_emotion_correct_responses = participant_scores.groupby(['Face Type', 'Emotion'])['Correct'].mean() * 100

    # Create a figure and specify a 2x2 GridSpec
    fig = plt.figure(figsize=(10, 8))
    gs = fig.add_gridspec(2, 2)  # 2 rows, 2 columns

    plt.suptitle('Correct Responses by Face Type, Emotion and Both, N = ' + str(len(participant_correct_responses)), fontsize=16)

    # Top-left: Face Type pie chart
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.pie(face_type_correct_responses,
            labels=face_type_correct_responses.index,
            autopct='%1.1f%%', startangle=90)
    ax1.set_title('Face Type\np = ' + str(p_value_face_type))

    # Top-right: Emotion pie chart
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.pie(emotion_correct_responses,
            labels=emotion_correct_responses.index,
            autopct='%1.1f%%', startangle=90)
    ax2.set_title('Emotion\np = ' + str(p_value_emotion))

    # Bottom row spanning both columns: Face Type & Emotion pie chart
    ax3 = fig.add_subplot(gs[1, :])  # span columns 0 and 1
    ax3.pie(face_type_emotion_correct_responses,
            labels=[f"{ft}-{em}" for ft, em in face_type_emotion_correct_responses.index],
            autopct='%1.1f%%', startangle=90)
    ax3.set_title('Face Type & Emotion\np = ' + str(p_value_interaction))

    plt.tight_layout()
    plt.savefig('plots/behavioural_responses/correct_responses_by_face_type_emotion.png', dpi=dpi / 4)
    plt.close()

    # get the average number of correct responses for each trial
    trial_correct_responses = participant_scores.groupby('Trial')['Correct'].mean().to_list()

    # calculate the regression line using stats.linregress
    x = range(1, len(trial_correct_responses) + 1)
    y = trial_correct_responses
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

    # plot this data
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(trial_correct_responses) + 1), trial_correct_responses, 'o-')
    plt.plot(x, [slope * i + intercept for i in x], 'r--')
    plt.xlabel('Trial')
    plt.ylabel('Percentage of Correct Responses')
    plt.title('Percentage of Correct Responses per Trial, R^2 = ' + str(round(r_value ** 2, 2)) + ', p = ' + str(round(p_value, 4)))
    plt.grid()
    plt.tight_layout()
    plt.savefig('plots/behavioural_responses/correct_responses_by_trial.png', dpi=dpi / 4)
    plt.close()

    # get the average response time for each trial
    trial_response_times = participant_scores.groupby('Trial')['Response Time'].mean().to_list()
    
    # calculate the regression line using stats.linregress
    x = range(1, len(trial_response_times) + 1)
    y = trial_response_times
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    
    # plot this data
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(trial_response_times) + 1), trial_response_times, 'o-')
    plt.plot(x, [slope * i + intercept for i in x], 'r--')
    plt.xlabel('Trial')
    plt.ylabel('Response Time (s)')
    plt.title('Response Time per Trial, R^2 = ' + str(round(r_value ** 2, 2)) + ', p = ' + str(p_value) + '\nCorrelation with Correct answer: ' + str(round(correlation, 4)))
    plt.grid()
    plt.tight_layout()
    plt.savefig('plots/behavioural_responses/response_times_by_trial.png', dpi=dpi / 4)
    plt.close()

# Mapping brain regions

In [ ]:
if get_mappings:
    raw_haemos = load_data('raw_haemos') if len(raw_haemos) == 0 else raw_haemos

    # get the channel names for hbo
    ch_names_hbo = [ch_name for ch_name in raw_haemos[0].ch_names if 'hbo' in ch_name]

    ch_mapping_hbo = {
        "Left Frontal": [],
        "Right Frontal": [],
        "Left Prefrontal": [],
        "Right Prefrontal": [],
        "Left Parietal": [],
        "Right Parietal": [],
        "Left Occipital": [],
        "Right Occipital": []
    }

    group_boundaries = [0]

    ch_mapping_hbo["Left Frontal"].append('S1_D1 hbo')

    ch_mapping_hbo["Left Frontal"].append('S1_D2 hbo')

    ch_mapping_hbo["Left Frontal"].append('S1_D17 hbo')

    # find the channels that have 'S2_', 'S3_', 'S4_', 'S5_' in them
    for ch_name in [ch_name for ch_name in ch_names_hbo if 'S2_' in ch_name or 'S3_' in ch_name or 'S4_' in ch_name or 'S5_' in ch_name]:
        ch_mapping_hbo["Left Frontal"].append(ch_name)

    ch_mapping_hbo["Left Frontal"].append('S6_D2 hbo')

    ch_mapping_hbo["Left Frontal"].append('S6_D3 hbo')

    ch_mapping_hbo["Left Frontal"].append('S6_D18 hbo')

    group_boundaries.append(len(ch_mapping_hbo["Left Frontal"]))

    # find the channels that have 'S9_', 'S10_', 'S11_', 'S12_' in them
    for ch_name in [ch_name for ch_name in ch_names_hbo if 'S9_' in ch_name or 'S10_' in ch_name or 'S11_' in ch_name or 'S12_' in ch_name]:
        ch_mapping_hbo["Right Frontal"].append(ch_name)

    group_boundaries.append(len(ch_mapping_hbo["Right Frontal"]) + group_boundaries[-1])

    ch_mapping_hbo["Left Prefrontal"].append('S6_D31 hbo')

    # find the channels that have 'S7_', 'S8_', 'S25_', 'S26_' in them
    for ch_name in [ch_name for ch_name in ch_names_hbo if 'S7_' in ch_name or 'S8_' in ch_name or 'S25_' in ch_name or 'S26_' in ch_name]:
        ch_mapping_hbo["Left Prefrontal"].append(ch_name)

    group_boundaries.append(len(ch_mapping_hbo["Left Prefrontal"]) + group_boundaries[-1])

    # find the channels that have 'S13_', 'S14_', 'S15_', 'S16_' in them
    for ch_name in [ch_name for ch_name in ch_names_hbo if 'S13_' in ch_name or 'S14_' in ch_name or 'S15_' in ch_name or 'S16_' in ch_name]:
        ch_mapping_hbo["Right Prefrontal"].append(ch_name)

    group_boundaries.append(len(ch_mapping_hbo["Right Prefrontal"]) + group_boundaries[-1])

    # find the channels that have 'S27_', 'S28_', 'S29_', 'S30_' in them
    for ch_name in [ch_name for ch_name in ch_names_hbo if 'S27_' in ch_name or 'S28_' in ch_name or 'S29_' in ch_name or 'S30_' in ch_name]:
        ch_mapping_hbo["Left Parietal"].append(ch_name)

    group_boundaries.append(len(ch_mapping_hbo["Left Parietal"]) + group_boundaries[-1])

    ch_mapping_hbo["Right Occipital"].append('S21_D13 hbo')

    ch_mapping_hbo["Right Occipital"].append('S21_D16 hbo')

    ch_mapping_hbo["Right Occipital"].append('S23_D15 hbo')

    ch_mapping_hbo["Right Occipital"].append('S23_D16 hbo')

    # find the channels that have 'S17_', 'S18_', 'S19_', 'S20_' in them
    for ch_name in [ch_name for ch_name in ch_names_hbo if 'S17_' in ch_name or 'S18_' in ch_name or 'S19_' in ch_name or 'S20_' in ch_name]:
        ch_mapping_hbo["Right Parietal"].append(ch_name)

    group_boundaries.append(len(ch_mapping_hbo["Right Parietal"]) + group_boundaries[-1])

    # find the channels that have 'S32_', 'S31_' in them
    for ch_name in [ch_name for ch_name in ch_names_hbo if 'S32_' in ch_name or 'S31_' in ch_name]:
        ch_mapping_hbo["Left Occipital"].append(ch_name)

    group_boundaries.append(len(ch_mapping_hbo["Left Occipital"]) + group_boundaries[-1])

    # find the channels that have 'S22_', 'S24_' in them
    for ch_name in [ch_name for ch_name in ch_names_hbo if 'S22_' in ch_name or 'S24_' in ch_name]:
        ch_mapping_hbo["Right Occipital"].append(ch_name)

    ch_mapping_hbo["Right Occipital"].append('S21_D28 hbo')

    ch_mapping_hbo["Right Occipital"].append('S23_D30 hbo')

    ch_mapping_hbr = {region: [channel.replace('hbo', 'hbr') for channel in ch_mapping_hbo[region]] for region in ch_mapping_hbo}

    ch_mapping_hbt = {region: [channel.replace('hbo', 'hbt') for channel in ch_mapping_hbo[region]] for region in ch_mapping_hbo}

    ch_mapping_all = {region: ch_mapping_hbo[region] + ch_mapping_hbr[region] + ch_mapping_hbt[region] for region in ch_mapping_hbo}

    # concatenate the values of the dictionary into a list
    all_channels_hbo = [channel for region in ch_mapping_hbo.values() for channel in region]

    # duplicate all_channels but replace 'hbo' with 'hbr'
    all_channels_hbr = [channel.replace('hbo', 'hbr') for channel in all_channels_hbo]

    all_channels_hbt = [channel.replace('hbo', 'hbt') for channel in all_channels_hbo]

    # concatenate all_channels_hbo and all_channels_hbr
    all_channels = all_channels_hbo + all_channels_hbr + all_channels_hbt

    # make a dictionary called ch_mapping_names that has the channel names without the 'hbo' or 'hbr' at the end
    ch_mapping_names = {region: [channel[:-4] for channel in ch_mapping_hbo[region]] for region in ch_mapping_hbo}

    # make a list of all the channel names without the 'hbo' or 'hbr' at the end
    all_channels_names = [channel[:-4] for channel in all_channels_hbo]

    ch_names_original = [ch_name[:-4] for ch_name in ch_names_hbo]

    # Collect all new variables into a dictionary
    mappings = {
        "ch_names_hbo": ch_names_hbo,
        "ch_mapping_hbo": ch_mapping_hbo,
        "ch_mapping_hbr": ch_mapping_hbr,
        "ch_mapping_hbt": ch_mapping_hbt,
        "ch_mapping_all": ch_mapping_all,
        "all_channels_hbo": all_channels_hbo,
        "all_channels_hbr": all_channels_hbr,
        "all_channels_hbt": all_channels_hbt,
        "all_channels": all_channels,
        "ch_mapping_names": ch_mapping_names,
        "all_channels_names": all_channels_names,
        "ch_names_original": ch_names_original,
        "group_boundaries": group_boundaries
    }

    # Save the dictionary to a JSON file
    with open("processed_data//mappings/mappings.json", "w") as json_file:
        json.dump(mappings, json_file, indent=4)

# Signal Quality

### Scalp Coupling Index (SCI)

In [ ]:
if get_full_sci:
    raw_ods = load_data('raw_ods') if len(raw_ods) == 0 else raw_ods

    # for each recording, count the number of channels with a sci greater than good_threshold
    good_channels = [sum(scalp_coupling_index(raw_od, verbose=False) >= sci_threshold) for raw_od in raw_ods]
    bad_channels = [sum(scalp_coupling_index(raw_od, verbose=False) < sci_threshold) for raw_od in raw_ods]
    good_recordings = sum([good_channel >= good_threshold * (good_channel + bad_channel) for good_channel, bad_channel in zip(good_channels, bad_channels)])

    # Plot the good vs bad channels for each recording in a dual bar chart
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.bar(range(len(good_channels)), good_channels, label='Good Channels')
    ax.bar(range(len(bad_channels)), bad_channels, bottom=good_channels, label='Bad Channels')
    ax.set_xlabel('Recording')
    ax.set_ylabel('Number of Channels')
    ax.axhline(raw_od.info['nchan'] * good_threshold, color='green', linestyle='--')
    title = f'Good vs Bad Channels (T = {sci_threshold})\nGood Recordings: {good_recordings}, N = {len(raw_ods)}, Retention Rate: {good_recordings / len(raw_ods) * 100:.2f}%'
    ax.set_title(title)
    ax.legend()
    plt.savefig(f'plots/signal quality/Signal Quality (SCI).png', dpi=dpi)
    plt.close()

### Coefficient of Variance (CV)

In [ ]:
if get_full_cv:
    raws = load_data('raws') if len(raws) == 0 else raws

    # for each recording, count the number of channels with a cv less than cov_threshold
    good_channels = [sum(100 * np.std(ch) / np.mean(ch) < cv_threshold for ch in get_long_channels(raw).get_data()) for raw in raws]
    bad_channels = [sum(100 * np.std(ch) / np.mean(ch) >= cv_threshold for ch in get_long_channels(raw).get_data()) for raw in raws]
    good_recordings = sum([good_channel >= good_threshold * (good_channel + bad_channel) for good_channel, bad_channel in zip(good_channels, bad_channels)])

    # Plot the good vs bad channels for each recording in a dual bar chart
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.bar(range(len(good_channels)), good_channels, label='Good Channels')
    ax.bar(range(len(bad_channels)), bad_channels, bottom=good_channels, label='Bad Channels')
    ax.set_xlabel('Recording')
    ax.set_ylabel('Number of Channels')
    ax.axhline(raw.info['nchan'] * good_threshold, color='green', linestyle='--')
    title = f'Good vs Bad Channels (T = {cv_threshold})\nGood Recordings: {good_recordings}, N = {len(raws)}, Retention Rate: {good_recordings / len(raws) * 100:.2f}%'
    ax.set_title(title)
    ax.legend()
    plt.savefig(f'plots/signal quality/Signal Quality (CV).png', dpi=dpi)
    plt.close()

### Get Peak Power/SCI Sliding Window CSV

In [2]:
if get_peak_power_sci_df:
    raw_ods = load_data('raw_ods') if len(raw_ods) == 0 else raw_ods

    peak_power_df = pd.DataFrame()
    sci_df = pd.DataFrame()
    for i, raw_od in enumerate(raw_ods, 1):
        raw_od_annotated_pp, scores_pp, times_pp = peak_power(raw_od, time_window=5, verbose=False)
        raw_od_annotated_sci, scores_sci, times_sci = scalp_coupling_index_windowed(raw_od, time_window=5, verbose=False)

        # Convert scores and times to a DataFrame
        df_pp = pd.DataFrame(scores_pp.T, columns=[ch_name for ch_name in raw_od.ch_names])
        df_sci = pd.DataFrame(scores_sci.T, columns=[ch_name for ch_name in raw_od.ch_names])

        # Add time window information
        df_pp["Start_Time"] = [t[0] for t in times_pp]
        df_pp["End_Time"] = [t[1] for t in times_pp]
        df_sci["Start_Time"] = [t[0] for t in times_sci]
        df_sci["End_Time"] = [t[1] for t in times_sci]

        # Add an index column for window number
        df_pp.insert(0, 'Window', range(1, len(df_pp) + 1))
        df_sci.insert(0, 'Window', range(1, len(df_sci) + 1))

        # Reorder columns so time comes first
        df_pp = df_pp[["Start_Time", "End_Time"] + list(df_pp.columns[:-2])]
        df_sci = df_sci[["Start_Time", "End_Time"] + list(df_sci.columns[:-2])]

        # remove the columns with '850' in the name
        df_pp = df_pp.loc[:, ~df_pp.columns.str.contains('850')]
        df_sci = df_sci.loc[:, ~df_sci.columns.str.contains('850')]

        # rename the columns to remove the ' 760' at the end if it exists
        df_pp.columns = [col[:-4] if col.endswith(' 760') else col for col in df_pp.columns]
        df_sci.columns = [col[:-4] if col.endswith(' 760') else col for col in df_sci.columns]

        # Add a column for participant number, make it the first column
        df_pp.insert(0, 'Participant', i)
        df_sci.insert(0, 'Participant', i)

        # Append the DataFrame to the list
        peak_power_df = pd.concat([peak_power_df, df_pp])
        sci_df = pd.concat([sci_df, df_sci])
        print(f"Processed participant {i}")

    # reset the index
    peak_power_df.reset_index(drop=True, inplace=True)
    sci_df.reset_index(drop=True, inplace=True)

    # Save the DataFrame to a CSV file
    peak_power_df.to_csv('processed_data/windows/peak_power.csv', index=False)
    sci_df.to_csv('processed_data/windows/sci.csv', index=False)

# Load the DataFrame
peak_power_df = pd.read_csv('processed_data/windows/peak_power.csv')
sci_df = pd.read_csv('processed_data/windows/sci.csv')

### Peak Power/SCI Sliding Window

In [23]:
if get_good_windows_plot:
    # Compute the proportion of windows with peak power > peak_power_threshold for each channel
    percentage_good_windows_peak_power_df = (
        peak_power_df.groupby("Participant")[peak_power_df.columns[4:]]
        .apply(lambda df: (df > peak_power_threshold).sum() / len(df))
    )

    # add a good recordings column
    good_recordings = (percentage_good_windows_peak_power_df > good_threshold).sum(axis=1) / len(percentage_good_windows_peak_power_df.columns)
    percentage_good_windows_peak_power_df.insert(0, f'Good Recordings (peak_power > {peak_power_threshold} for > {good_threshold * 100}% of channels)', good_recordings)

    # Compute the proportion of windows with SCI > good_threshold for each channel
    percentage_good_windows_sci_df = (
        sci_df.groupby("Participant")[sci_df.columns[4:]]
        .apply(lambda df: (df > sci_threshold).sum() / len(df))
    )

    # add a good recordings column
    good_recordings = (percentage_good_windows_sci_df > good_threshold).sum(axis=1) / len(percentage_good_windows_sci_df.columns)
    percentage_good_windows_sci_df.insert(0, f'Good Recordings (SCI > {sci_threshold} for > {good_threshold * 100}% of channels)', good_recordings)

    # merge the two dataframes on the first 2 columns
    percentage_good_windows_df = pd.merge(percentage_good_windows_peak_power_df[percentage_good_windows_peak_power_df.columns[:1]], percentage_good_windows_sci_df[percentage_good_windows_sci_df.columns[:1]], on='Participant')

    # create a new column that is true if both columns are greater than good_threshold
    percentage_good_windows_df['Good Recording'] = (percentage_good_windows_df.iloc[:, 0] > good_threshold) & (percentage_good_windows_df.iloc[:, 1] > good_threshold)

    # save the dataframe to a csv file
    percentage_good_windows_df.to_csv('processed_data/windows/percentage_good_windows.csv', index=False)

    # Plot a bar chart where only the SCI windows are shown for each participant
    fig, ax = plt.subplots(figsize=(8, 6))
    x = np.arange(len(percentage_good_windows_df))
    ax.bar(x, percentage_good_windows_df.iloc[:, 1], label='SCI', color='#FF8C00')  # Darker orange
    ax.axhline(good_threshold, color='green', linestyle='--')
    ax.set_xlabel('Participant')
    ax.set_ylabel('Percentage of Channels (%)')
    ax.set_title(f'Percentage of Channels where SCI > ' + str(sci_threshold) + ' for > ' + str(good_threshold * 100) + f'% of windows\nGood Recordings: {percentage_good_windows_df["Good Recording"].sum()}/{len(percentage_good_windows_df)}, Retention Rate: {percentage_good_windows_df["Good Recording"].sum() / len(percentage_good_windows_df) * 100:.2f}%')
    plt.tight_layout(pad=0)
    plt.savefig(f'plots/signal quality/Percentage of Good Windows.png', dpi=dpi // 2)
    plt.close()

### Peak Power/SCI Over Time

In [ ]:
if get_slope_pp_sci_plot:
    pp_channel_slopes = []
    sci_channel_slopes = []
    for pp_channel, sci_channel in zip(percentage_good_windows_peak_power_df.columns[4:], percentage_good_windows_sci_df.columns[4:]):
        pp_slope = []
        sci_slope = []
        for participant in peak_power_df['Participant'].unique():
            # get the data for the channel
            pp_array = peak_power_df[peak_power_df['Participant'] == participant][pp_channel]
            sci_array = sci_df[sci_df['Participant'] == participant][sci_channel]

            # get a line of best fit for the data
            x = np.arange(len(pp_array))
            pp_m, pp_b = np.polyfit(x, pp_array, 1)
            pp_slope.append(pp_m)

            sci_m, sci_b = np.polyfit(x, sci_array, 1)
            sci_slope.append(sci_m)
        pp_channel_slopes.append((pp_channel, np.mean(pp_slope)))
        sci_channel_slopes.append((sci_channel, np.mean(sci_slope)))

    # plot the slopes
    fig, ax = plt.subplots(figsize=(20, 6))
    # sort the channels by slope
    pp_channel_slopes.sort(key=lambda x: x[1])
    sci_channel_slopes.sort(key=lambda x: x[1])
    bar_width = 0.45
    x = np.arange(len(pp_channel_slopes))
    ax.bar(x, [slope for channel, slope in pp_channel_slopes], bar_width, label='Peak Power')
    ax.bar(x + bar_width, [slope for channel, slope in sci_channel_slopes], bar_width, label='SCI')
    ax.set_xlabel('Channel')
    ax.set_ylabel('Slope')
    ax.set_title('Slope of Peak Power and SCI over Time')
    ax.set_xticks(x + bar_width / 2)
    ax.set_xticklabels([channel for channel, slope in pp_channel_slopes])
    ax.legend()
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(f'plots/signal quality/Slope of Peak Power and SCI over Time.png', dpi=dpi)
    plt.close()

### Head Size vs. SCI Windows

In [ ]:
if get_head_size_vs_sci_plot:
    raw_haemos = load_data('raw_haemos') if len(raw_haemos) == 0 else raw_haemos

    cap_size = 58
    percentage_good_windows_sci_df_with_head_size = percentage_good_windows_sci_df.copy()

    # add the head size to the percentage_good_windows_sci_df as the first column
    if 'Head Size (cm)' not in percentage_good_windows_sci_df_with_head_size.columns:
        percentage_good_windows_sci_df_with_head_size.insert(0, 'Head Size (cm)', cap_size)

    no_head_size = []

    for i, raw_haemo in enumerate(raw_haemos, 1):
        # get the head size
        head_size = get_info(raw_haemo)['remarks']
        if head_size:
            head_size = float(head_size) * 2.54
        else:
            # add the participant number to the no_head_size list
            no_head_size.append(i)
            continue
        
        # append the head_size to percentage_good_windows_sci_df
        percentage_good_windows_sci_df_with_head_size.loc[i, 'Head Size (cm)'] = head_size

    # remove the participants with no head size
    percentage_good_windows_sci_df_with_head_size = percentage_good_windows_sci_df_with_head_size.drop(no_head_size)

    # get the correlation between head size and the channels
    correlations = []
    for channel in percentage_good_windows_sci_df_with_head_size.columns[2:]:
        correlation = percentage_good_windows_sci_df_with_head_size['Head Size (cm)'].corr(percentage_good_windows_sci_df_with_head_size[channel])
        correlations.append((channel, correlation))

    # get the correlation between head size and the second column
    correlation = percentage_good_windows_sci_df_with_head_size['Head Size (cm)'].corr(percentage_good_windows_sci_df_with_head_size.iloc[:, 1])

    # plot the correlations
    fig, ax = plt.subplots(figsize=(20, 6))
    # sort the channels by correlation
    correlations.sort(key=lambda x: x[1])
    bar_width = 0.45
    x = np.arange(len(correlations))
    ax.bar(x, [correlation for channel, correlation in correlations], bar_width)
    ax.set_xlabel('Channel')
    ax.set_ylabel('Correlation')
    ax.set_title('Correlation between Head Size and SCI, Correlation with Good Recordings: ' + str(correlation) + ', N = ' + str(len(percentage_good_windows_sci_df_with_head_size)))
    ax.set_xticks(x)
    ax.set_xticklabels([channel for channel, correlation in correlations])
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(f'plots/signal quality/Correlation between Head Size and SCI.png', dpi=dpi / 3)
    plt.close()

### Average SCI per Channel across Participants

In [ ]:
if get_across_participant_sci_plots:
    # get a list of participants in percentage_good_windows_sci_df where Good Recording is True
    good_participants = percentage_good_windows_df[percentage_good_windows_df['Good Recording'] == True].index

    # make a dataframe of the average sci for each channel
    avg_sci_df = sci_df.groupby('Participant').mean().drop(columns=['Window', 'Start_Time', 'End_Time'])

    # drop the participants that are not in good_participants
    avg_sci_df_good = avg_sci_df.loc[good_participants]

    # drop the participants that are in good_participants
    avg_sci_df_bad = avg_sci_df.drop(index=good_participants)

    # make a list of the dataframes
    avg_sci_dfs = [avg_sci_df, avg_sci_df_good, avg_sci_df_bad]
    df_names = ['All Participants', 'Good Participants', 'Bad Participants']
    color_list = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'pink', 'gray']

    for df, df_name in zip(avg_sci_dfs, df_names):

        # make a violin plot of the average sci for each channel
        fig, ax = plt.subplots(figsize=(35, 6))
        parts = ax.violinplot(df, showmeans=False, widths=1, showextrema=False)

        # match the violin plot colors to the columns in avg_sci_df to the channels in ch_mapping_names
        color_i = 0
        colors = []
        region_labels = []

        # for each region in ch_mapping_names, apply the color to the channels in that region
        for region, channels in mappings['ch_mapping_names'].items():
            for channel in channels:
                if channel in df.columns:
                    colors.append(color_list[color_i])
                    region_labels.append(region)
            color_i += 1

        # set the colors of the violins
        for i, pc in enumerate(parts['bodies']):
            pc.set_facecolor(colors[i])
            pc.set_edgecolor('black')
            pc.set_alpha(1)

        # create a legend
        handles = [plt.Rectangle((0, 0), 1, 1, color=color) for color in list(dict.fromkeys(colors))]
        ax.legend(handles, list(dict.fromkeys(region_labels)), loc='lower left')

        # add a white scatter plot of the mean sci for each channel
        ax.scatter(np.arange(1, len(df.columns) + 1), df.mean(), color='white', zorder=3)

        ax.set_xlabel('Channel')
        ax.set_ylabel('Average SCI')
        ax.set_ylim(0, 1)
        ax.axhline(good_threshold, color='green', linestyle='--')
        ax.set_title(f'Average SCI per Channel: ({df_name}), N = {len(df)}')
        ax.set_xticks(np.arange(1, len(df.columns) + 1))
        ax.set_xticklabels(df.columns)
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.savefig(f'plots/signal quality/Average SCI (Windowed) per Channel/{df_name}.png', dpi=dpi / 3)
        plt.close()

### SCI of Windows per Channel for each Participant

In [ ]:
if get_participant_sci_plots:
    for i in sci_df['Participant'].unique():
        df = sci_df[sci_df['Participant'] == i]

        # get the good recording status
        good_recording = False
        if percentage_good_windows_df['Good Recording'][i]:
            good_recording = True

        # make a violin plot of the sci for each channel
        fig, ax = plt.subplots(figsize=(35, 6))
        parts = ax.violinplot(df[df.columns[4:]], showmeans=False, widths=1, showextrema=False)

        # match the violin plot colors to the columns in sci_df to the channels in ch_mapping_names
        color_i = 0
        colors = []
        region_labels = []

        # for each region in ch_mapping_names, apply the color to the channels in that region
        for region, channels in mappings['ch_mapping_names'].items():
            for channel in channels:
                if channel in df.columns:
                    colors.append(color_list[color_i])
                    region_labels.append(region)
            color_i += 1

        # set the colors of the violins
        for j, pc in enumerate(parts['bodies']):
            pc.set_facecolor(colors[j])
            pc.set_edgecolor('black')
            pc.set_alpha(1)

        # create a legend
        handles = [plt.Rectangle((0, 0), 1, 1, color=color) for color in list(dict.fromkeys(colors))]
        ax.legend(handles, list(dict.fromkeys(region_labels)), loc='lower left')

        # add a white scatter plot of the mean sci for each channel
        ax.scatter(np.arange(1, len(df.columns) - 3), df[df.columns[4:]].mean(), color='white', zorder=3)

        ax.set_xlabel('Channel')
        ax.set_ylabel('SCI')
        ax.set_ylim(0, 1)
        ax.axhline(good_threshold, color='green', linestyle='--')
        ax.set_title(f'SCI per Channel: Participant {i}, Windows = {len(df)}, Good Recording = {good_recording}')
        ax.set_xticks(np.arange(1, len(df.columns) - 3))
        ax.set_xticklabels(df.columns[4:])
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.savefig(f'plots/signal quality/Average SCI (Windowed) per Channel/individual/Participant {i}.png', dpi=dpi / 3)
        plt.close()

# Get Epoch Data

In [9]:
modes = ['face_type', 'emotion']
conditions_list = {
    'face_type': ['Real', 'Virt'],
    'emotion': ['Joy', 'Fear', 'Anger', 'Disgust', 'Sadness', 'Neutral', 'Surprise']
}
channel_types = ['hbo', 'hbr', 'hbt']

if get_epochs:
    raw_haemo_good_recordings = load_data('raw_haemos', 'good') if len(raw_haemo_good_recordings) == 0 else raw_haemo_good_recordings
    for mode in modes:
        for condition in conditions_list[mode]:
            participants = []
            for raw_haemo in raw_haemo_good_recordings:
                ind_epochs = relabel_annotations(raw_haemo.copy(), mode=mode)[condition]
                channels = []
                for channel_type in channel_types:
                    epoch_channel = pick_channels(ind_epochs, channel_type)
                    channels.append(epoch_channel.get_data())

                participants.append(np.array(channels))
            participants = np.array(participants)
            participants = np.moveaxis(participants, 1, 2)
            np.save(f'processed_data/epochs/{mode}_{condition}.npy', participants)

# We now have epochs[mode][condition].shape = (n_participants, n_epochs, n_channel_types, n_channels, n_times)
epochs = {
    mode: {
        condition: np.load(f'processed_data/epochs/{mode}_{condition}.npy') 
        if os.path.exists(f'processed_data/epochs/{mode}_{condition}.npy') 
        else None
        for condition in conditions_list[mode]
    }
    for mode in modes
}

dict_channel_types = {
    channel_type: idx for idx, channel_type in enumerate(channel_types)
}

### Face Type/Emotion Epochs

In [ ]:
modes = ['face_type_emotion']
conditions_list = {
    'face_type': ['Real', 'Virt'],
    'emotion': ['Joy', 'Fear', 'Anger', 'Disgust', 'Sadness', 'Neutral', 'Surprise'],
}
conditions_list['face_type_emotion'] = [''.join(pair) for pair in itertools.product(conditions_list['face_type'], conditions_list['emotion'])]
channel_types = ['hbo', 'hbr', 'hbt']

if get_face_type_emotion_epochs:
    raw_haemo_good_recordings = load_data('raw_haemos', 'good') if len(raw_haemo_good_recordings) == 0 else raw_haemo_good_recordings
    for mode in modes:
        for condition in conditions_list[mode]:
            all_condition_list = []
            for i, raw_haemo in enumerate(raw_haemo_good_recordings, 1):
                ind_epochs = relabel_annotations(raw_haemo.copy(), mode=mode)

                if condition not in list(set(ind_epochs.annotations.description)):
                    print(f'Participant {i} missing {condition}')
                    continue

                ind_epochs = ind_epochs[condition]

                channels = []
                for channel_type in channel_types:
                    epoch_channel = pick_channels(ind_epochs, channel_type)
                    channels.append(epoch_channel.get_data())

                all_condition_list.append(np.array(channels))
                    
            all_conds = None
            for cond in all_condition_list:
                if all_conds is None:
                    all_conds = cond
                else:
                    all_conds = np.concatenate((all_conds, cond), axis=1)
            all_conds = np.moveaxis(all_conds, 0, 1)
            np.save(f'processed_data/epochs/{mode}_{condition}.npy', all_conds)

# We now have face_emotion_epochs[mode][condition].shape = (n_epochs, n_channel_types, n_channels, n_times)
face_emotion_epochs = {
    mode: {
        condition: np.load(f'processed_data/epochs/{mode}_{condition}.npy') 
        if os.path.exists(f'processed_data/epochs/{mode}_{condition}.npy') 
        else None
        for condition in conditions_list[mode]
    }
    for mode in modes
}

# GLM analysis

In [5]:
modes = ['all', 'face_type', 'emotion', 'neutral_vs_emotion']
conditions_list = {
    'all': ['Blck', 'Base'],
    'face_type': ['Real', 'Virt', 'Base'],
    'emotion': ['Joy', 'Fear', 'Anger', 'Disgust', 'Sadness', 'Neutral', 'Surprise'],
    'neutral_vs_emotion': ['Emotion', 'Neutral', 'Base']
}
channel_types = ['hbo', 'hbr', 'hbt']

if get_glm_analysis:
    raw_haemo_good_recordings = load_data('raw_haemos', 'good') if len(raw_haemo_good_recordings) == 0 else raw_haemo_good_recordings
    for mode in modes:
        cha_df = pd.DataFrame()
        roi_df = pd.DataFrame()
        con_df = pd.DataFrame()
        for i, raw_haemo in enumerate(raw_haemo_good_recordings, 1):
            raw_haemo_annots = pick_channels(raw_haemo, channel_types)
            relabel_annotations(raw_haemo_annots, mode=mode)

            # Create a design matrix
            design_matrix = make_first_level_design_matrix(
                raw_haemo_annots,
                drift_model="cosine",
                high_pass=0.03125,  # The cutoff period (1/high_pass) should be set as the longest period between two trials of the same condition multiplied by 2
                hrf_model="spm",
                stim_dur=16.0,
            )
            
            # Run GLM
            glm_est = run_glm(raw_haemo_annots, design_matrix, n_jobs=n_jobs)

            cha = glm_est.to_dataframe()

            # in ch_mapping_all, for each list of channels in the dict, each string is formatted as 'S{number}_D{number} {hbo/hbr}', extract the number from the string and replace the string with [number, number]
            groups = {region: [[int(re.findall(r'\d+', channel)[0]), int(re.findall(r'\d+', channel)[1])] for channel in mappings['ch_mapping_all'][region]] for region in mappings['ch_mapping_all']}
            # apply picks_pair_to_idx to each region in groups
            for region in groups:
                groups[region] = picks_pair_to_idx(raw_haemo_annots, groups[region], on_missing='ignore')

            # Compute region of interest results from channel data
            roi = glm_est.to_dataframe_region_of_interest(
                groups, design_matrix.columns, demographic_info=True
            )

            # Define contrasts
            contrast_matrix = np.eye(design_matrix.shape[1])
            basic_conts = dict(
                [(column, contrast_matrix[j]) for j, column in enumerate(design_matrix.columns)]
            )
            contrasts = []
            unique_annots = np.unique(raw_haemo_annots.annotations.description).tolist()
            pairs = list(itertools.combinations(unique_annots, 2))
            # include the opposite of each pair
            pairs = pairs + [(pair[1], pair[0]) for pair in pairs]

            # Compute defined contrast pairs
            for pair in pairs:
                con = glm_est.compute_contrast(basic_conts[pair[0]] - basic_conts[pair[1]]).to_dataframe()
                con["contrast_pair"] = f"{pair[0]} - {pair[1]}"
                contrasts.append(con)

            # Add the participant ID to the dataframes
            roi["Participant"] = cha["Participant"] = i
            for con in contrasts:
                con["Participant"] = i

            # Convert to uM for nicer plotting below.
            cha["theta"] = [t * 1.0e6 for t in cha["theta"]]
            roi["theta"] = [t * 1.0e6 for t in roi["theta"]]
            for con in contrasts:
                con["effect"] = [t * 1.0e6 for t in con["effect"]]

            # Append the dataframes to the main dataframes
            cha_df = pd.concat([cha_df, cha])
            roi_df = pd.concat([roi_df, roi])
            for con in contrasts:
                con_df = pd.concat([con_df, con])

        # Apply FDR correction to all dataframes
        for df in [cha_df, roi_df, con_df]:
            if 'p_value' in df.columns:
                _, df['p_value_fdr'] = fdrcorrection(df['p_value'])

        cha_df.to_csv('processed_data/glm/cha/cha_df_' + mode + '.csv', index=False)
        roi_df.to_csv('processed_data/glm/roi/roi_df_' + mode + '.csv', index=False)
        con_df.to_csv('processed_data/glm/cons/con_df_' + mode + '.csv', index=False)

# load the dataframes
glm = {
    mode: {
        'cha': pd.read_csv(f'processed_data/glm/cha/cha_df_{mode}.csv'),
        'roi': pd.read_csv(f'processed_data/glm/roi/roi_df_{mode}.csv'),
        'con': pd.read_csv(f'processed_data/glm/cons/con_df_{mode}.csv')
    }
    for mode in modes
}

### Individual GLM Results

In [7]:
if get_ind_glm_plots:
    channel_type = ['hbt']
    for mode in modes:
        grp_results = glm[mode]['roi'].query(f"Condition in {conditions_list[mode]}")
        grp_results = grp_results.query(f"Chroma in {channel_types}")

        theta_min = grp_results['theta'].min()
        theta_max = grp_results['theta'].max()

        # clear any files in the plots/glm/individual folder
        for f in os.listdir('plots/glm/individual_' + mode):
            os.remove(os.path.join('plots/glm/individual_' + mode, f))

        for i in grp_results['Participant'].unique():
            fig, ax = plt.subplots(figsize=(12, 6))
            fig.suptitle(f'GLM Results for Participant {i}')

            sns.swarmplot(data=grp_results.query(f"Participant == {i} and Chroma == '{channel_type[0]}'"),
                  x='Condition', y='theta', hue='ROI', ax=ax, dodge=False)
            ax.set_title(f'{channel_type[0]}')
            ax.set_ylabel('Theta (uM)')
            ax.set_ylim(theta_min, theta_max)
            ax.set_xlabel('Condition')
            ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

            plt.tight_layout(rect=[0, 0, 1, 0.95])
            plt.savefig(f'plots/glm/individual_{mode}/Participant {i}.png', dpi=dpi / 4)
            plt.close()

### Group GLM Results

In [8]:
if get_group_glm_plots:
    channel_type = ['hbt']
    for mode in modes:
        grp_results = glm[mode]['roi'].query(f"Condition in {conditions_list[mode]}")
        grp_results = grp_results.query(f"Chroma in {channel_type}")

        # Run a GLM model
        roi_model = mixedlm("theta ~ -1 + ROI:Condition:Chroma", grp_results, groups=grp_results["Participant"]).fit(method="nm")

        # Get the results of the model and put it in a csv file
        roi_model_results = statsmodels_to_results(roi_model)

        # Apply FDR correction to the p-values in roi_model_results
        _, roi_model_results['P>|z|_fdr'] = fdrcorrection(roi_model_results['P>|z|'])

        # plot the results of the model for 'hbt' channel type
        fig, ax = plt.subplots(figsize=(12, 6))
        fig.suptitle(f'GLM Results for Group - {mode}')

        sns.swarmplot(data=roi_model_results.query(f"Chroma == '{channel_type[0]}'"),
              x='Condition', y='Coef.', hue='ROI', ax=ax, dodge=False)
        ax.set_title(f'{channel_type[0]}')
        ax.set_ylabel('Theta (uM)')
        ax.set_xlabel('Condition')
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.savefig(f'plots/glm/group_results/results_{mode}.png', dpi=dpi / 4)
        plt.close()

### Group Contrasts

In [9]:
if get_group_contrast_plots:

    # pick channel type
    channel_type = 'hbt'

    for mode in modes:
        pairs = list(itertools.combinations(conditions_list[mode], 2))
        for pair in pairs:
            con_summary = glm[mode]['con'].query(f"contrast_pair == '{pair[0]} - {pair[1]}'")

            if len(con_summary) == 0:
                print(f"No data for contrast {pair[0]} - {pair[1]}")
                continue

            raw_haemos = load_data('raw_haemos') if len(raw_haemos) == 0 else raw_haemos
            raw_haemo = pick_channels(raw_haemos[0].copy(), 'hbo')
            raw_haemo_chs = pick_channels(raw_haemos[0].copy(), channel_type).ch_names

            con_summary_channel = con_summary.query(f"Chroma in {[channel_type]}")

            # Run group level model and convert to dataframe
            con_model = mixedlm("effect ~ -1 + ch_name:Chroma", con_summary_channel, groups=con_summary_channel["Participant"]).fit(method="nm")

            # Get the results of the model
            con_model_df = statsmodels_to_results(con_model, raw_haemo_chs)

            
            # Apply FDR correction to the group-level p-values from the model
            _, con_model_df['p_value_fdr'] = fdrcorrection(con_model_df['P>|z|'])

            # Plot the topographic map
            fig, axes = plt.subplots(1, 2, figsize=(10, 6))
            fig.suptitle(f'Contrast: {pair[0]} > {pair[1]}, {channel_type}')

            # set the "Coef." value to near 0 for channels that are not significant
            coef = con_model_df.copy()
            coef.loc[coef['p_value_fdr'] > 0.05, 'Coef.'] /= 15

            norm = mpl.colors.Normalize(vmin=-3, vmax=3)

            # plot the topomap
            mne.viz.plot_topomap(coef["Coef."].to_numpy(), raw_haemo.info, extrapolate='head', show=False, axes=axes[0], sensors='ok', cnorm=norm)

            # make a plot of a colorbar without using plt.colorbar
            cb1 = mpl.colorbar.ColorbarBase(axes[1], cmap=mpl.cm.RdBu_r, norm=norm, orientation='vertical')
            cb1.set_label('Coefficient Value', fontsize=10)
            cb1.ax.set_aspect(3)

            plt.tight_layout(pad=0, rect=[0, 0, 1, 0.95])
            if "Neutral" in pair:
                plt.savefig(f'plots/glm/contrasts/differences_neutral/Contrast_{pair[0]}-{pair[1]}.png', dpi=dpi / 2)
            else:
                plt.savefig(f'plots/glm/contrasts/differences/Contrast_{pair[0]}-{pair[1]}.png', dpi=dpi / 2)
            plt.close()

# ROI Timeseries Activity

In [ ]:
if get_roi_timeseries_activity:
    raw_haemo_good_recordings = load_data('raw_haemos', 'good') if len(raw_haemo_good_recordings) == 0 else raw_haemo_good_recordings

    channel_types = ['hbo', 'hbr', 'hbt']
    
    # create an empty dataframe
    roi_timeseries_activity = pd.DataFrame()

    tmin = 4
    tmax = 16

    i = 1
    for raw_haemo in raw_haemo_good_recordings:
        face_epochs = relabel_annotations(raw_haemo.copy(), mode='face_type')
        emotion_epochs = relabel_annotations(raw_haemo.copy(), mode='emotion')

        # crop the epochs to tmin-tmax
        face_epochs.crop(tmin=tmin, tmax=tmax)

        # convert the epochs to a dataframe
        face_epochs_df = face_epochs.to_data_frame()

        # remove the baseline condition
        face_epochs_df = face_epochs_df.where(face_epochs_df['condition'] != 'Base').dropna()

        # average these columns: column_names[3:].tolist(), by the epoch column and condition column
        face_epochs_df = face_epochs_df.groupby(['epoch', 'condition']).mean().reset_index()

        # crop the epochs to tmix-tmax
        emotion_epochs.crop(tmin=tmin, tmax=tmax)

        # convert the epochs to a dataframe
        emotion_epochs_df = emotion_epochs.to_data_frame()

        # remove the baseline condition
        emotion_epochs_df = emotion_epochs_df.where(emotion_epochs_df['condition'] != 'Base').dropna()

        # average these columns: column_names[3:].tolist(), by the epoch column and condition column
        emotion_epochs_df = emotion_epochs_df.groupby(['epoch', 'condition']).mean().reset_index()

        # add the condition column from the emotion_epochs_df to the face_epochs_df and line it up with the epoch column
        face_epochs_df['emotion'] = emotion_epochs_df['condition']

        # put the emotion column in the third column
        all_epochs_df = face_epochs_df[['epoch', 'condition', 'emotion'] + face_epochs_df.columns[2:-1].tolist()]

        # rename the condition column to face type
        all_epochs_df.rename(columns={'condition': 'face type'}, inplace=True)

        # divide the epoch column by 2 and floor it and convert it to an integer
        all_epochs_df['epoch'] = (all_epochs_df['epoch'] // 2).astype(int)

        # remove the time column
        all_epochs_df.drop(columns='time', inplace=True)

        if 'hbo' in channel_types:
            for region, channels in mappings['ch_mapping_hbo'].items():
                # Ensure the channels exist in the dataframe to avoid errors
                valid_channels = [channel for channel in channels if channel in all_epochs_df.columns]
                if valid_channels:
                    # Create a new column for the region's average
                    all_epochs_df[region + ' Average Hbo'] = all_epochs_df[valid_channels].mean(axis=1)

        if 'hbr' in channel_types:
            for region, channels in mappings['ch_mapping_hbr'].items():
                # Ensure the channels exist in the dataframe to avoid errors
                valid_channels = [channel for channel in channels if channel in all_epochs_df.columns]
                if valid_channels:
                    # Create a new column for the region's average
                    all_epochs_df[region + ' Average Hbr'] = all_epochs_df[valid_channels].mean(axis=1)

        if 'hbt' in channel_types:
            for region, channels in mappings['ch_mapping_hbt'].items():
                # Ensure the channels exist in the dataframe to avoid errors
                valid_channels = [channel for channel in channels if channel in all_epochs_df.columns]
                if valid_channels:
                    # Create a new column for the region's average
                    all_epochs_df[region + ' Average Hbt'] = all_epochs_df[valid_channels].mean(axis=1)

        # drop all the channel columns
        all_epochs_df.drop(columns=mappings['all_channels'], inplace=True)

        # add participant number column
        all_epochs_df['Participant'] = i

        # make the participant number the first column
        all_epochs_df = all_epochs_df[['Participant'] + all_epochs_df.columns[:-1].tolist()]

        # add measurement date column
        #all_epochs_df['Measurement Date'] = raw_haemo.info['meas_date']

        # add an empty column for repetition and put it after the emotion column
        all_epochs_df.insert(4, 'Repetition', '')

        conditions = defaultdict(int)
        for index, row in all_epochs_df.iterrows():
            # add the condition-emotion pair to the conditions dictionary and increment the count
            conditions[f"{row['face type']}-{row['emotion']}"] += 1

            # add the count to the repetition column
            all_epochs_df.at[index, 'Repetition'] = conditions[f"{row['face type']}-{row['emotion']}"]

        # put it after the repetition column
        all_epochs_df.insert(5, 'Sex', '')

        # add the sex column
        all_epochs_df['Sex'] = get_info(raw_haemo)['gender']

        # add the dataframe to the average_timeseries_activity datafram
        roi_timeseries_activity = pd.concat([roi_timeseries_activity, all_epochs_df])

        i += 1

    # reset the index
    roi_timeseries_activity.reset_index(drop=True, inplace=True)

    # name the index column to 'observation'
    roi_timeseries_activity.index.name = 'Observation'

    # replace any spaces in the column names with underscores
    roi_timeseries_activity.columns = roi_timeseries_activity.columns.str.replace(' ', '_')

    # capitalize the column names
    roi_timeseries_activity.columns = roi_timeseries_activity.columns.str.capitalize()

    mappings = {}
    for col in roi_timeseries_activity.select_dtypes(include=['object']).columns:
        # Create a mapping dictionary for the column
        unique_values = roi_timeseries_activity[col].unique()
        col_mapping = {val: idx for idx, val in enumerate(unique_values)}
        mappings[col] = col_mapping
        
        # Replace the column values in the DataFrame with numeric values
        roi_timeseries_activity[col] = roi_timeseries_activity[col].map(col_mapping)

    # Save mappings to a JSON file
    with open('processed_data/roi_timeseries_activity/mappings.json', 'w') as json_file:
        json.dump(mappings, json_file, indent=4)

    # Get the unique number of participants
    num_participants = roi_timeseries_activity['Participant'].nunique()

    # save the dataframe to a csv file
    roi_timeseries_activity.to_csv(f'processed_data/roi_timeseries_activity/roi_timeseries_activity_sci{sci_threshold}_n{num_participants}.csv')

# ERP Plots

In [24]:
modes = ['face_type', 'emotion']
conditions_list = {
    'face_type': ['Real', 'Virt'],
    'emotion': ['Joy', 'Fear', 'Anger', 'Disgust', 'Sadness', 'Neutral', 'Surprise']
}
channel_types = ['hbo', 'hbr', 'hbt']

if get_erp_plots:
    # make a color list for the different channel types
    colors = {
        'hbo': 'red',
        'hbr': 'blue',
        'hbt': 'purple'
    }

    # pick a channel type to plot the difference between regions
    difference_channel_type = 'hbt'

    for mode in modes:
        for condition in conditions_list[mode]:

            fig, ax = plt.subplots(figsize=(12, 6))

            for channel_type in channel_types:
                # get epoch data, shape is now (n_participants, n_times)
                epoch_data = np.mean(epochs[mode][condition][:, :, dict_channel_types[channel_type]], axis=(1, 2))
                epoch_data_mean = np.mean(epoch_data, axis=0)
                
                # get the standard error of the mean
                epoch_data_sem = np.std(epoch_data, axis=0) / np.sqrt(epoch_data.shape[0])

                # plot the mean
                ax.plot(epoch_data_mean, label=channel_type, color=colors[channel_type])

                # plot the standard error of the mean as a shaded area
                ax.fill_between(np.arange(epoch_data_mean.shape[0]), epoch_data_mean - epoch_data_sem, epoch_data_mean + epoch_data_sem, color=colors[channel_type], alpha=0.3)
            
            ax.set_xlabel('Time (s)')
            ax.set_ylabel('uM')
            ax.set_title(f'ERP Plot for {mode}: {condition}, N = {len(epochs[mode][condition])}, Shaded Area: SEM')
            ax.legend()
            ax.set_ylim(-3e-6, 3e-6) # set the ylims to -+3 * 10^-6
            
            # set the xticks to the time points, 0-16 seconds, instead of 99 time points
            ax.set_xticks(np.arange(0, 99, 6))
            ax.set_xticklabels(np.arange(0, 17, 1))
            plt.tight_layout()
            plt.savefig(f'plots/erp/erp_conditions/{condition}.png', dpi=dpi / 2)
            plt.close()

        for cond1, cond2 in itertools.combinations(conditions_list[mode], 2):
            # create a 8x1 subplot that shares the x-axis and i can iterate over
            fig, axes = plt.subplots(8, 1, figsize=(12, 24), sharex=True)        

            # in ch_mapping_all, for each list of channels in the dict, filter for the channels with difference_channel_type in the string
            groups = {region: [channel for channel in channels if difference_channel_type in channel] for region, channels in mappings['ch_mapping_all'].items()}
            group_i = 0
            for region, ch_name in groups.items():
                # Get the indices for the current region from group_boundaries
                keys_list = list(groups.keys())
                start_idx = mappings['group_boundaries'][keys_list.index(region)]
                end_idx = mappings['group_boundaries'][keys_list.index(region) + 1] if keys_list.index(region) + 1 < len(mappings['group_boundaries']) else None

                # get the data for the current region
                epoch_data_region1 = np.mean(epochs[mode][cond1][:, :, dict_channel_types[difference_channel_type], start_idx:end_idx], axis=(0, 1))
                epoch_data_region2 = np.mean(epochs[mode][cond2][:, :, dict_channel_types[difference_channel_type], start_idx:end_idx], axis=(0, 1))

                # get the mean of the data
                epoch_data_region1_mean = np.mean(epoch_data_region1, axis=0)
                epoch_data_region2_mean = np.mean(epoch_data_region2, axis=0)

                # get the standard error of the mean
                epoch_data_region1_sem = np.std(epoch_data_region1, axis=0) / np.sqrt(epoch_data_region1.shape[0])
                epoch_data_region2_sem = np.std(epoch_data_region2, axis=0) / np.sqrt(epoch_data_region2.shape[0])

                # plot the mean
                axes[group_i].plot(epoch_data_region1_mean, label=cond1, color='red')
                axes[group_i].plot(epoch_data_region2_mean, label=cond2, color='blue')

                # plot the standard error of the mean as a shaded area
                axes[group_i].fill_between(np.arange(epoch_data_region1_mean.shape[0]), epoch_data_region1_mean - epoch_data_region1_sem, epoch_data_region1_mean + epoch_data_region1_sem, color='red', alpha=0.3)
                axes[group_i].fill_between(np.arange(epoch_data_region2_mean.shape[0]), epoch_data_region2_mean - epoch_data_region2_sem, epoch_data_region2_mean + epoch_data_region2_sem, color='blue', alpha=0.3)

                # set the title of the subplot to the region
                if group_i == 0:
                    axes[group_i].set_title(f'ERP Plot for {mode}: {cond1} - {cond2}, N = {len(epochs[mode][cond1])}, Shaded Area: SEM, {difference_channel_type}\n{region}')
                else:
                    axes[group_i].set_title(region)
                
                # set the ylims to -+7 * 10^-6
                axes[group_i].set_ylim(-7e-6, 7e-6)

                # add a legend to the subplot
                axes[group_i].legend()

                group_i += 1

            axes[-1].set_xlabel('Time (s)')
            axes[-1].set_xticks(np.arange(0, 99, 6))
            axes[-1].set_xticklabels(np.arange(0, 17, 1))
            axes[-1].set_ylabel('uM')
            plt.tight_layout()
            if "Neutral" == cond1 or "Neutral" == cond2:
                plt.savefig(f'plots/erp/erp_differences_neutral/{cond1}_{cond2}.png', dpi=dpi / 2)
            else:
                plt.savefig(f'plots/erp/erp_differences/{cond1}_{cond2}.png', dpi=dpi / 2)
            plt.close()

# Connectivity Analysis

In [2]:
modes = ['all', 'face_type', 'emotion']
conditions_list = {
    'all': ['Blck', 'Base'],
    'face_type': ['Real', 'Virt', 'Base'],
    'emotion': ['Joy', 'Fear', 'Anger', 'Disgust', 'Sadness', 'Neutral', 'Surprise']
}

connectivity_method = [spectral_connectivity_time, spectral_connectivity_epochs]

# pick the channels
channel_types = ['hbo', 'hbr', 'hbt']

# main channel type
channel_type = 'hbt'

# pick the connectivity method
method = "coh"

# pick the mode
con_mode = "cwt_morlet"

# pick the frequency range
cwt_freqs = np.linspace(0.1, 0.5, 5)

# pick the number of cycles
cwt_n_cycles = 1

# average the connectivity matrices across frequencies
faverage = True

if run_ind_connectivity:
    raw_haemo_good_recordings = load_data('raw_haemos', 'good') if len(raw_haemo_good_recordings) == 0 else raw_haemo_good_recordings
    for func in connectivity_method:
        for mode in modes:
            for condition in conditions_list[mode]:
                # create an empty list to store the connectivity for each participant
                participant_cons = []

                # for each raw_haemo in raw_haemo_good_recordings, compute the connectivity
                for i, raw_haemo in enumerate(raw_haemo_good_recordings, 1):

                    # relabel the annotations and pick the condition
                    ind_epochs = relabel_annotations(raw_haemo.copy(), mode=mode)[condition]

                    # create an empty list to store each channel type's connectivity
                    cons = []

                    # for each channel type in channel_types
                    for channel_type in channel_types:
                        # pick the channels
                        epochs_channel = pick_channels(ind_epochs, channel_type)
                    
                        # use func to compute the connectivity
                        if func.__name__ == 'spectral_connectivity_epochs':
                            con = func(
                                epochs_channel,
                                method=method,
                                mode=con_mode,
                                cwt_freqs=cwt_freqs,
                                cwt_n_cycles=cwt_n_cycles,
                                faverage=faverage,
                                n_jobs=n_jobs,
                                verbose=False
                            )

                            data = con.get_data()
                            data = np.moveaxis(np.moveaxis(data, 1, 2), 0, 1)
                        elif func.__name__ == 'spectral_connectivity_time':
                            con = func(
                                epochs_channel,
                                method=method,
                                mode=con_mode,
                                freqs=cwt_freqs,
                                n_cycles=cwt_n_cycles,
                                faverage=faverage,
                                n_jobs=n_jobs,
                                verbose=False
                            )

                            data = con.get_data()
                        else:
                            raise ValueError('func must be spectral_connectivity_epochs or spectral_connectivity_time')

                        # append the connectivity to the cons list
                        cons.append(data)

                    participant_cons.append(np.array(cons))
                    
                # save the connectivity to disk
                np.save(f'processed_data\\{func.__name__}\\individual_cons\\{mode}_{condition}_con.npy', np.array(participant_cons))

        # make a dictionary to store the connectivity parameters
        ind_connectivity_params = {
            "func": func.__name__,
            "channel_types": channel_types,
            "method": method,
            "con_mode": con_mode,
            "cwt_freqs": cwt_freqs.tolist(),
            "cwt_n_cycles": cwt_n_cycles,
            "faverage": faverage,
            "ch_names": pick_channels(raw_haemo, channel_types).ch_names
        }

        # save the connectivity parameters to disk in preprocessed_data\connectivity
        with open(f"processed_data\\{func.__name__}\\ind_connectivity_params.json", "w") as f:
            json.dump(ind_connectivity_params, f)

# load the numpy files from disk so we have ind_con[func_name][mode][condition].shape = (n_participants, n_channel_types, n_epochs OR n_times, n_channels, n_freqs
ind_con = {
    func.__name__: {
        mode: {
            condition: np.load(f'processed_data\\{func.__name__}\\individual_cons\\{mode}_{condition}_con.npy')
            for condition in conditions_list[mode]
        }
        for mode in modes
    }
    for func in connectivity_method
}

ind_connectivity_params = {
    func.__name__: json.load(open(f"processed_data\\{func.__name__}\\ind_connectivity_params.json", "r"))
    for func in connectivity_method
}

dict_channel_types = {
    channel_type: idx for idx, channel_type in enumerate(channel_types)
}

participants = ind_con[connectivity_method[0].__name__][modes[0]][conditions_list[modes[0]][0]].shape[0]

### Condition Connectivity Heatmap/Chord Plots

In [3]:
modes = ['face_type', 'emotion']
conditions_list = {
    'face_type': ['Real', 'Virt', 'Base'],
    'emotion': ['Joy', 'Fear', 'Anger', 'Disgust', 'Sadness', 'Neutral', 'Surprise']
}

if get_condition_con_plots:
    # make a color scheme
    colorscheme = dict(
        facecolor='white',
        textcolor='black',
        colormap='hot',
        facecolor2='black',
        textcolor2='white',
    )

    # get the node angles
    node_angles = circular_layout(
        mappings['ch_names_original'], mappings['all_channels_names'], start_pos=90, group_boundaries=mappings['group_boundaries']
    )

    for func in connectivity_method:
        for mode in modes:
            for condition in conditions_list[mode]:
                averaged_data = np.mean(ind_con[func.__name__][mode][condition][:, dict_channel_types[channel_type]], axis=(0, 1, 3))
                
                # Get the grid size
                grid_size = int(np.sqrt(averaged_data.size))

                # Reshape the data into a 2D grid
                heatmap_data = averaged_data.reshape((grid_size, grid_size))

                # Make the matrix symmetric
                symmetric_data = heatmap_data + heatmap_data.T

                # Set the diagonal to the highest value
                np.fill_diagonal(symmetric_data, np.max(symmetric_data))

                # Plot the heatmap
                fig, ax = plt.subplots(figsize=(25, 25))
                im = ax.imshow(symmetric_data, cmap='viridis')
                ax.set_title(f'Connectivity Heatmap for {func.__name__}, Mode: {mode}, Condition: {condition}, Channel Type: {channel_type}')
                ax.set_xlabel('Channel')
                ax.set_ylabel('Channel')
                ax.set_xticks(np.arange(grid_size))
                ax.set_yticks(np.arange(grid_size))
                ch_names = [ch_name for ch_name in ind_connectivity_params[func.__name__]['ch_names'] if channel_type in ch_name]
                ax.set_xticklabels(ch_names)
                ax.set_yticklabels(ch_names)
                plt.setp(ax.get_xticklabels(), rotation=90, ha="right", rotation_mode="anchor")

                cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
                cbar.set_label('Connectivity Value')
                plt.savefig(f'plots/{func.__name__}/heatmaps/conditions/{mode}_{condition}_con.png', dpi=dpi / 4)
                plt.close()

                # find the min of the averaged_data, when dropping all the 0s
                min_val = np.min(averaged_data[np.nonzero(averaged_data)])

                # find the max of the averaged_data, when dropping all the 0s
                max_val = np.max(averaged_data[np.nonzero(averaged_data)])

                # Plot the connectivity circle
                plot_connectivity_circle(
                    heatmap_data,
                    node_names=mappings['ch_names_original'],
                    node_angles=node_angles,
                    n_lines=10000,
                    title=f'{func.__name__}, {condition}, {channel_type}',
                    colorbar_size=1,
                    fontsize_colorbar=12,
                    facecolor=colorscheme['facecolor'],
                    textcolor=colorscheme['textcolor'],
                    colormap=colorscheme['colormap'],
                    padding=3,
                    vmin=min_val,
                    vmax=max_val,
                    fontsize_title=12,
                    colorbar=True,
                    show=False
                )
                plt.savefig(f'plots/{func.__name__}/chord_plots/conditions/{mode}_{condition}_con.png', dpi=dpi / 4)
                plt.close()

### Condition Connectivity Heatmaps Across Time GIFS

In [4]:
if get_condition_con_gif_plots:
    for mode in modes:
        for condition in conditions_list[mode]:
            # the shape of averaged_data is (n_times, n_channels)
            averaged_data = ind_con['spectral_connectivity_epochs'][mode][condition][:, dict_channel_types[channel_type], :, :, 0].mean(axis=0)

            # reshape the data to (n_times, sqrt(n_channels), sqrt(n_channels))
            averaged_data = averaged_data.reshape(averaged_data.shape[0], int(np.sqrt(averaged_data.shape[1])), int(np.sqrt(averaged_data.shape[1])))

            # make the matrix symmetric for the (sqrt(n_channels), sqrt(n_channels)) part
            for matrix in averaged_data:
                matrix += matrix.T
                np.fill_diagonal(matrix, np.max(matrix))

            # create a funcanimation object to animate the connectivity across time
            fig, ax = plt.subplots(figsize=(12, 10))
            im = ax.imshow(averaged_data[0], cmap='viridis', animated=True)
            ax.set_title(f'{mode} {condition} {channel_type} - Frame 1/{averaged_data.shape[0]}')
            ax.set_xlabel('Channel')
            ax.set_ylabel('Channel')

            plt.colorbar(im)
            plt.tight_layout()

            def update(frame):
                im.set_array(averaged_data[frame])
                ax.set_title(f'{mode} {condition} {channel_type} - Time Point: {frame + 1}/{averaged_data.shape[0]}')
                return im,

            ani = FuncAnimation(fig, update, frames=averaged_data.shape[0], blit=True)
            ani.save(f'plots/spectral_connectivity_epochs/heatmaps/gifs/{mode}_{condition}.gif', writer='pillow', fps=8, dpi=dpi / 4)
            plt.close(fig)

### Condition Connectivity Variance Across Participants

In [5]:
if get_variance_con_plots:
    for mode in modes:
        vmax = None
        for condition in conditions_list[mode]:
            # the shape of averaged_data is (n_participants, n_channels)
            averaged_data = ind_con['spectral_connectivity_time'][mode][condition][:, dict_channel_types[channel_type], :, :, 0].mean(axis=1)

            # reshape the data to (n_participants, n_channels, n_channels)
            averaged_data = averaged_data.reshape(averaged_data.shape[0], int(np.sqrt(averaged_data.shape[1])), int(np.sqrt(averaged_data.shape[1])))

            # make the matrix symmetric for the (sqrt(n_channels), sqrt(n_channels)) part
            for matrix in averaged_data:
                matrix += matrix.T
                np.fill_diagonal(matrix, np.max(matrix))

            # create a matrix of the variance of each channel pair over all participants
            variance_matrix = np.var(averaged_data, axis=0)

            # Plot the heatmap
            fig, ax = plt.subplots(figsize=(10, 10))
            im = ax.imshow(variance_matrix, cmap='viridis')
            ax.set_title(f'Variance Heatmap for Mode: {mode}, Condition: {condition}, Channel Type: {channel_type}')
            ax.set_xlabel('Channel')
            ax.set_ylabel('Channel')
            
            cbar = plt.colorbar(im, ax=ax)
            cbar.set_label('Variance Value')
            if vmax is None:
                vmax = variance_matrix.max()
            
            im.set_clim(0, vmax)
            plt.tight_layout()

            plt.savefig(f'plots/spectral_connectivity_time/heatmaps/variance/{mode}_{condition}_con.png', dpi=dpi / 4)
            plt.close()

### Group Level Connectivity t-tests

In [6]:
connectivity_method = [spectral_connectivity_time, spectral_connectivity_epochs]

modes = ['face_type', 'emotion']
conditions_list = {
    'face_type': ['Real', 'Virt', 'Base'],
    'emotion': ['Joy', 'Fear', 'Anger', 'Disgust', 'Sadness', 'Neutral', 'Surprise']
}

if run_group_level_t_tests:
    for func in connectivity_method:
        for mode in modes:
            for condition1, condition2 in itertools.combinations(conditions_list[mode], 2):
                # Get the data for the two conditions
                data1 = np.mean(ind_con[func.__name__][mode][condition1][:, dict_channel_types[channel_type]], axis=(1, 3))
                data2 = np.mean(ind_con[func.__name__][mode][condition2][:, dict_channel_types[channel_type]], axis=(1, 3))

                # reshape the data to (num_participants, int(np.sqrt(num_channel_connections)), int(np.sqrt(num_channel_connections)))
                data1 = data1.reshape(data1.shape[0], int(np.sqrt(data1.shape[1])), int(np.sqrt(data1.shape[1])))
                data2 = data2.reshape(data2.shape[0], int(np.sqrt(data2.shape[1])), int(np.sqrt(data2.shape[1])))

                # ensure the data shapes are the same
                assert data1.shape == data2.shape

                # ensure the data is not empty or contains NaNs
                assert not np.isnan(data1).any()
                assert not np.isnan(data2).any()

                # apply fisher z transform to the data
                data1 = np.arctanh(data1)
                data2 = np.arctanh(data2)

                # Initialize matrices to store t and p-values
                t_vals = np.zeros(data1.shape[1:])
                p_values = np.zeros(data1.shape[1:])

                # Iterate over each pair of channels
                for i in range(data1.shape[1]):
                    for j in range(i): # because the matrix is lower triangular
                        # Run the t-test
                        t_val, p_val = stats.ttest_rel(data1[:, i, j], data2[:, i, j])

                        # Store the results
                        t_vals[i, j] = t_val
                        p_values[i, j] = p_val

                # Flatten the lower triangular valid p-values (non-nan) for FDR correction
                valid_idx = ~np.isnan(p_values)
                p_values_flat = p_values[valid_idx]

                # Correct p-values for multiple comparisons
                _, p_values_flat_fdr = fdrcorrection(p_values_flat)

                # Reshape the corrected p-values back to matrix form
                p_values_fdr = p_values_flat_fdr.reshape(p_values.shape)

                # Save the p-values to disk
                np.save(f'processed_data\\{func.__name__}\\paired_t_tests\\{mode}_{condition1}_{condition2}_p_values.npy', p_values_fdr)

# Load the p-values from disk
group_level_t_tests_p_values = {
    func.__name__: {
        mode: {
            f"{condition1}_{condition2}": np.load(f'processed_data\\{func.__name__}\\paired_t_tests\\{mode}_{condition1}_{condition2}_p_values.npy')
            for condition1, condition2 in itertools.combinations(conditions_list[mode], 2)
        }
        for mode in modes
    }
    for func in connectivity_method
}

### Group Level Connectivity t-test Chord Plots


In [7]:
connectivity_method = [spectral_connectivity_time, spectral_connectivity_epochs]
modes = ['face_type', 'emotion']
conditions_list = {
    'face_type': ['Real', 'Virt', 'Base'],
    'emotion': ['Joy', 'Fear', 'Anger', 'Disgust', 'Sadness', 'Neutral', 'Surprise']
}

if get_group_level_t_tests_chord_plots:

    # make a color scheme
    colorscheme = dict(
        facecolor='white',
        textcolor='black',
        colormap='hot',
        facecolor2='black',
        textcolor2='white',
    )

    # get the node angles
    node_angles = circular_layout(
        mappings['ch_names_original'], mappings['all_channels_names'], start_pos=90, group_boundaries=mappings['group_boundaries']
    )
    
    for func in connectivity_method:
        for mode in modes:
            for condition1, condition2 in itertools.combinations(conditions_list[mode], 2):
                p_values = group_level_t_tests_p_values[func.__name__][mode][f"{condition1}_{condition2}"]

                # apply –log₁₀(p) to the p-values
                p_values = -np.log10(p_values)

                # set any p-values that are < -log₁₀(0.05) to 0
                p_values = np.where(p_values < -np.log10(0.05), 0, p_values)

                # Set the diagonal to 0
                np.fill_diagonal(p_values, 0)

                plot_connectivity_circle(
                    p_values,
                    node_names=mappings['ch_names_original'],
                    node_angles=node_angles,
                    n_lines=10000,
                    title=f'{func.__name__}, {condition1} - {condition2}, {channel_type}, -log₁₀(p)',
                    colorbar_size=1,
                    fontsize_colorbar=12,
                    facecolor=colorscheme['facecolor'],
                    textcolor=colorscheme['textcolor'],
                    colormap=colorscheme['colormap'],
                    padding=3,
                    vmin=0,
                    vmax=-np.log10(0.0005),
                    fontsize_title=16,
                    colorbar=True,
                    show=False
                )
                plt.savefig(f'plots/{func.__name__}/chord_plots/group_level_t_tests/{mode}_{condition1}_{condition2}_p_values.png', dpi=dpi / 4)
                plt.close()

C:\Users\super\AppData\Local\Temp\ipykernel_26972\3809924952.py:30: RuntimeWarning: divide by zero encountered in log10
  p_values = -np.log10(p_values)


### Group Level Connectivity t-test ROI Chord Plots
Remember to edit the plot_connectivity_circle function to include the diagonals: \mne\viz\circle.py

In [8]:
modes = ['face_type', 'emotion']
conditions_list = {
    'face_type': ['Real', 'Virt', 'Base'],
    'emotion': ['Joy', 'Fear', 'Anger', 'Disgust', 'Sadness', 'Neutral', 'Surprise']
}

if get_group_level_t_tests_roi_chord_plots:
    # Get the names of the regions
    regions = list(mappings['ch_mapping_names'].keys())

    colorscheme = dict(
        facecolor='white',
        textcolor='black',
        colormap='Reds',
        facecolor2='black',
        textcolor2='white',
    )

    # get the node angles
    node_angles = circular_layout(
        regions, regions, start_pos=90
    )

    # First, flatten the dictionary into a list of tuples (channel_name, region)
    flat_list = [(channel, region) for region, channels in mappings['ch_mapping_names'].items() for channel in channels]

    # Now, replace the channel name with its enumeration (starting from 0)
    reshaped_list = [(i, region) for i, (_, region) in enumerate(flat_list)]

    for func in connectivity_method:
        for mode in modes:
            for condition1, condition2 in itertools.combinations(conditions_list[mode], 2):
                p_values_by_roi = {}

                for i, row in enumerate(group_level_t_tests_p_values[func.__name__][mode][f'{condition1}_{condition2}']):
                    region_i = reshaped_list[i][1]
                    for j, element in enumerate(row):
                        region_j = reshaped_list[j][1]

                        # skip the zeros
                        if element == 0:
                            continue

                        # Store the significant p-values for each ROI pair
                        if f'{region_i}_{region_j}' not in p_values_by_roi:
                            p_values_by_roi[f'{region_i}_{region_j}'] = [element]
                        else:
                            p_values_by_roi[f'{region_i}_{region_j}'].append(element)

                for roi_pair, p_values in p_values_by_roi.items():
                    # filter out the p-values that are > 0.05
                    p_values_by_roi[roi_pair] = [p_value for p_value in p_values if p_value < 0.05]

                    # Get the count of p-values for each ROI pair
                    p_values_by_roi[roi_pair] = len(p_values_by_roi[roi_pair])

                # Create a matrix filled with NaNs
                matrix = np.full((len(regions), len(regions)), np.nan)

                # Fill the matrix using the dictionary values
                for key, value in p_values_by_roi.items():
                    r1, r2 = key.split('_')
                    i = regions.index(r1)
                    j = regions.index(r2)
                    # Fill the lower triangular part of the matrix
                    if i > j:
                        matrix[i, j] = value
                    else:
                        matrix[j, i] = value

                # replace any 0s with NaNs
                matrix[matrix == 0] = np.nan

                # Set the number of discrete colors to the highest number of significant p-values
                discrete_cmap = cm.get_cmap('Reds', np.nanmax(matrix) + 1)

                # Plot the connectivity circle
                plot_connectivity_circle(
                    matrix,
                    node_names=regions,
                    node_angles=node_angles,
                    n_lines=10000,
                    title=f'Number of significant p_values for {func.__name__}\n{mode}, {condition1} - {condition2}',
                    colorbar_size=1,
                    fontsize_colorbar=12,
                    node_width=40,
                    linewidth=4.5,
                    facecolor=colorscheme['facecolor'],
                    textcolor=colorscheme['textcolor'],
                    colormap=discrete_cmap,
                    padding=3,
                    fontsize_title=12,
                    fontsize_names=10,
                    colorbar=True,
                    show=False
                )
                plt.savefig(f'plots/{func.__name__}/chord_plots/group_level_t_tests_roi/{mode}_{condition1}_{condition2}.png', dpi=dpi / 4)
                plt.close()

C:\Users\super\AppData\Local\Temp\ipykernel_26972\3747612962.py:75: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  discrete_cmap = cm.get_cmap('Reds', np.nanmax(matrix) + 1)


# Decoding Analysis

## Get Data

In [3]:
modes = ['all', 'face_type', 'emotion']

conditions_list = {
    'all': ['Blck', 'Base'],
    'face_type': ['Real', 'Virt'],
    'emotion': ['Joy', 'Fear', 'Anger', 'Disgust', 'Sadness', 'Neutral', 'Surprise']
}

channel_types = ['hbo', 'hbr', 'hbt']

dict_channel_types = {
    channel_type: idx for idx, channel_type in enumerate(channel_types)
}

channel_type = 'hbo'

raw_haemo_good_recordings = load_data('raw_haemos', 'good') if len(raw_haemo_good_recordings) == 0 else raw_haemo_good_recordings

func = spectral_connectivity_time

# load the numpy files from disk so we have ind_con[func_name][mode][condition].shape = (n_participants, n_channel_types, n_epochs, n_channels)
ind_con = {
    mode: {
        condition: np.load(f'processed_data\\{func.__name__}\\individual_cons\\{mode}_{condition}_con.npy')
        for condition in conditions_list[mode]
    }
    for mode in modes
}

# Example hyperparameter grids for each model
param_grids = {
    'RandomForestClassifier': {
        'randomforestclassifier__n_estimators': [100, 200, 300, 400, 500],
    },
    'LogisticRegression': {
        'logisticregression__C': [0.01, 0.1, 1, 10, 100]
    },
    'LGBMClassifier': {
        'lgbmclassifier__n_estimators': [100, 200, 300, 400, 500],
    },
    'KNeighborsClassifier': {
        'kneighborsclassifier__n_neighbors': [3, 5, 7, 9, 11],
    },
    'MLPClassifier': {
        'mlpclassifier__hidden_layer_sizes': [(50, 50), (100, 50), (100, 100)],
    },
    'QuadraticDiscriminantAnalysis': {
        'quadraticdiscriminantanalysis__reg_param': [0.0, 0.1, 0.5, 1.0],
    },
    'LinearDiscriminantAnalysis': {
        'lineardiscriminantanalysis__shrinkage': [None, 'auto', 0.1, 0.5, 1.0]
    },
    'GaussianNB': {
        'gaussiannb__var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]
    }
}

## Within Participants

### Using Raw Data

#### Traditional Models

In [ ]:
if run_traditional_raw_within_decoding:
    models_to_run = [
        RandomForestClassifier(random_state=42, n_jobs=n_jobs),
        LogisticRegression(random_state=42, n_jobs=n_jobs),
        LGBMClassifier(random_state=42, n_jobs=n_jobs),
        KNeighborsClassifier(n_jobs=n_jobs),
        MLPClassifier(random_state=42, verbose=False),
        QuadraticDiscriminantAnalysis(),
        LinearDiscriminantAnalysis(),
        GaussianNB()
    ]

    for model in models_to_run:
        model_score = {mode: None for mode in modes}
        for mode in modes:
            st_scores = []
            for raw_haemo in raw_haemo_good_recordings:
                # Relabel the annotations
                ind_epochs = relabel_annotations(raw_haemo.copy(), mode=mode)[conditions_list[mode]]

                # pick the channels
                ind_epochs = pick_channels(ind_epochs, channel_type)

                # Get the data and labels
                X = ind_epochs.get_data(copy=False)
                y = ind_epochs.events[:, 2]

                # Reshape the data to (n_epochs, n_channels * n_times)
                X = X.reshape(X.shape[0], -1)

                # Standardize the data
                X = StandardScaler().fit_transform(X)
                
                # Compute the cross-validated scores for the model
                # Use StratifiedKFold with 5 splits
                scores = 100 * cross_val_score(
                    model, 
                    X,
                    y, 
                    cv=5,
                    n_jobs=n_jobs,
                    scoring='accuracy',
                    verbose=0
                )

                # Append the scores to the list
                st_scores.append(np.mean(scores))

            # Return a dictionary with each score, and the mean score
            model_score[mode] = {
                'participant_scores': st_scores,
                'mean_score': np.mean(st_scores)
            }

        # Save the model scores to disk
        with open(f'processed_data/models/raw_within_scores/{model.__class__.__name__}_scores.json', 'w') as f:
            json.dump(model_score, f)

### Using Connectivity Data

#### Traditional Models

In [ ]:
if run_traditional_con_within_decoding:
    models_to_run = [
        RandomForestClassifier(random_state=42, n_jobs=n_jobs),
        LogisticRegression(random_state=42, n_jobs=n_jobs),
        LGBMClassifier(random_state=42, n_jobs=n_jobs),
        KNeighborsClassifier(n_jobs=n_jobs),
        MLPClassifier(random_state=42, verbose=False),
        QuadraticDiscriminantAnalysis(),
        LinearDiscriminantAnalysis(),
        GaussianNB()
    ]

    for model in models_to_run:
        model_score = {mode: None for mode in modes}
        for mode in modes:
            st_scores = []
            for i in range(ind_con[modes[0]][conditions_list[modes[0]][0]].shape[0]):
                participant_conds = []
                labels = []

                for condition in conditions_list[mode]:
                    x = ind_con[mode][condition][i, dict_channel_types[channel_type], :, :, 0]
                    label = np.array([condition] * x.shape[0])
                    participant_conds.append(x)
                    labels.append(label)
                
                # stack the data by the first axis
                X = np.vstack(participant_conds)
                
                # stack the labels by the first axis
                y = np.hstack(labels)

                # map the labels to integers
                label_mapping = {label: idx for idx, label in enumerate(np.unique(y))}
                y = np.array([label_mapping[label] for label in y])

                # Standardize the data
                X = StandardScaler().fit_transform(X)

                # Compute the cross-validated scores for the model
                # Use StratifiedKFold with 5 splits
                scores = 100 * cross_val_score(
                    model, 
                    X,
                    y, 
                    cv=5,
                    n_jobs=n_jobs,
                    scoring='accuracy',
                    verbose=0
                )
            
                # Append the scores to the list
                st_scores.append(np.mean(scores, axis=0))
            
            # Return a dictionary with each score, and the mean score
            model_score[mode] = {
                'participant_scores': st_scores,
                'mean_score': np.mean(st_scores)
            }

        # save the model scores to disk
        with open(f'processed_data/models/con_within_scores/{model.__class__.__name__}_scores.json', 'w') as f:
            json.dump(model_score, f)

## Across Participants

### Using Raw Data

#### Traditional Models

In [ ]:
if run_traditional_raw_across_decoding:
    models_to_run = [
        RandomForestClassifier(random_state=42, n_jobs=n_jobs),
        LogisticRegression(random_state=42, n_jobs=n_jobs),
        LGBMClassifier(random_state=42, n_jobs=n_jobs),
        KNeighborsClassifier(n_jobs=n_jobs),
        MLPClassifier(random_state=42, verbose=False),
        QuadraticDiscriminantAnalysis(),
        LinearDiscriminantAnalysis(),
        GaussianNB()
    ]

    # Outer cross-validation: leave-one-group-out
    logo = LeaveOneGroupOut()

    for model in models_to_run:
        model_score = {mode: {} for mode in modes}
        model_name = model.__class__.__name__
        param_grid = param_grids.get(model_name, {})

        for mode in modes:
            X_list, y_list, groups_list = [], [], []
            for i, raw_haemo in enumerate(raw_haemo_good_recordings, 1):
                # Relabel annotations and select the channel according to mode
                ind_epochs = relabel_annotations(raw_haemo.copy(), mode=mode)[conditions_list[mode]]
                ind_epochs = pick_channels(ind_epochs, channel_type)
                # Get data (n_epochs x n_channels x n_times) and labels
                X = ind_epochs.get_data(copy=False)
                y = ind_epochs.events[:, 2]
                X_list.append(X)
                y_list.append(y)
                # Create a group array indicating participant index for each epoch
                groups_list.append(np.full(len(y), i))

            # Concatenate data across participants and reshape to (n_epochs, n_channels * n_times)
            X = np.concatenate(X_list, axis=0)
            y = np.concatenate(y_list, axis=0)
            groups = np.concatenate(groups_list, axis=0)
            X = X.reshape(X.shape[0], -1)

            outer_scores = []
            outer_best_params = []

            # Outer CV loop
            for train_index, test_index in logo.split(X, y, groups):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]
                groups_train = groups[train_index]  # Use groups for inner CV

                # Pipeline: scale features then fit model
                pipe = make_pipeline(StandardScaler(), model)

                # Inner CV using GroupKFold (note: no shuffling available)
                inner_cv = GroupKFold(n_splits=5)

                # Hyperparameter search using GridSearchCV
                rs = GridSearchCV(
                    estimator=pipe,
                    param_grid=param_grid,
                    cv=inner_cv,
                    scoring='accuracy',
                    n_jobs=n_jobs,
                    verbose=0
                )
                # IMPORTANT: pass groups for the inner CV
                rs.fit(X_train, y_train, groups=groups_train)

                # Evaluate best estimator on the outer test fold
                test_score = 100 * rs.best_estimator_.score(X_test, y_test)
                outer_scores.append(test_score)
                outer_best_params.append(rs.best_params_)

            # Save per-mode results
            model_score[mode] = {
                'logo_scores': outer_scores,
                'mean_score': np.mean(outer_scores),
                'fold_best_params': outer_best_params
            }

        # Write results for the current model to disk
        with open(f'processed_data/models/raw_across_scores/{model_name}_scores.json', 'w') as f:
            json.dump(model_score, f)

#### LSTM Model

In [ ]:
modes = ['face_type']
raw_haemos = load_data('raw_haemos') if len(raw_haemos) == 0 else raw_haemos

if run_dl_raw_across_decoding:

    logo = LeaveOneGroupOut()

    model_score = {mode: None for mode in modes}
    for mode in modes:
        X_list, y_list, groups_list = [], [], []
        for i, raw_haemo in enumerate(raw_haemos, 1):
            # Relabel the annotations
            ind_epochs = relabel_annotations(raw_haemo.copy(), mode=mode)[conditions_list[mode]]
            
            # Select the channel(s)
            ind_epochs = pick_channels(ind_epochs, channel_type)
            
            # Get the data and labels:
            # X shape: (n_epochs, n_channels, n_times)
            X = ind_epochs.get_data()
            y = ind_epochs.events[:, 2]
            
            X_list.append(X)
            y_list.append(y)
            # Assign a group ID (participant index) for each epoch
            groups_list.append(np.full(len(y), i))
        
        # Concatenate data across participants
        X = np.concatenate(X_list, axis=0)
        y = np.concatenate(y_list, axis=0)

        # Map labels to integers
        label_mapping = {label: idx for idx, label in enumerate(np.unique(y))}
        y = np.array([label_mapping[label] for label in y])

        # Create a group array identifying the participant index for each epoch
        groups = np.concatenate(groups_list, axis=0)

        # New shape: (n_epochs, n_times, n_channels)
        X = np.transpose(X, (0, 2, 1))

        # Flatten epochs and time for scaling, then reshape back.
        X_reshaped = X.reshape(-1, X.shape[-1])
        X_reshaped = StandardScaler().fit_transform(X_reshaped)

        # Reshape back to 3D
        X = X_reshaped.reshape(X.shape)

        print(X.shape)

        # Leave-One-Group-Out Cross Validation:
        fold_scores = []
        for train_idx, test_idx in logo.split(X, y, groups=groups):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            # Define a CNN model for the current fold
            model = tf.keras.Sequential([
                tf.keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2])),
                tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
                tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)),
                tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2)),
                tf.keras.layers.Dense(128, activation='relu'),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(64, activation='relu'),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
            ])

            model.compile(
                optimizer='sgd',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
            )

            # Train the model on the training fold
            history = model.fit(
                X_train,
                y_train,
                epochs=100,
                batch_size=32,
                validation_split=0.2,
                callbacks=[
                    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
                    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5)
                ],
                verbose=1
            )

            # Evaluate on the test fold (the left-out group)
            score = model.evaluate(X_test, y_test, verbose=0)
            plot_history(history, mode, score[1])
            break
    #         fold_scores.append(100 * score[1])

    #     # Return a dictionary with each score, and the mean score
    #     model_score[mode] = {
    #         'logo_scores': fold_scores,
    #         'mean_score': np.mean(fold_scores)
    #     }

    # # Save the model scores to disk
    # with open(f'processed_data/models/raw_across_scores/{model.__class__.__name__}_scores.json', 'w') as f:
    #     json.dump(model_score, f)

### Using Connectivity Data

#### Traditional Models

In [ ]:
if run_traditional_con_across_decoding:
    models_to_run = [
        RandomForestClassifier(random_state=42, n_jobs=n_jobs),
        LogisticRegression(random_state=42, n_jobs=n_jobs),
        LGBMClassifier(random_state=42, n_jobs=n_jobs),
        KNeighborsClassifier(n_jobs=n_jobs),
        MLPClassifier(random_state=42, verbose=False),
        QuadraticDiscriminantAnalysis(),
        LinearDiscriminantAnalysis(),
        GaussianNB()
    ]

    # Outer CV: leave-one-group-out (e.g. leave-one-participant-out)
    logo = LeaveOneGroupOut()

    # Loop over models
    for model in models_to_run:
        model_score = {mode: {} for mode in modes}
        model_name = model.__class__.__name__
        # Get hyperparameter grid (if available) for this model
        param_grid = param_grids.get(model_name, {})

        # Loop over decoding modes
        for mode in modes:
            X_list, y_list, groups_list = [], [], []
            # Build feature matrices from connectivity data per participant
            for i in range(ind_con[modes[0]][conditions_list[modes[0]][0]].shape[0]):
                participant_conds = []
                labels = []
                for condition in conditions_list[mode]:
                    # Here we select the connectivity data for a specific channel type and condition.
                    # The shape of x is assumed to be: n_epochs x ... (we take the first connectivity matrix)
                    x = ind_con[mode][condition][i, dict_channel_types[channel_type], :, :, 0]
                    # Create labels for each epoch corresponding to this condition
                    label = np.array([condition] * x.shape[0])
                    participant_conds.append(x)
                    labels.append(label)
                # Stack epochs for the current participant
                X_part = np.vstack(participant_conds)
                y_part = np.hstack(labels)
                # Map string labels to integers consistently across participants
                label_mapping = {label: idx for idx, label in enumerate(np.unique(y_part))}
                y_part = np.array([label_mapping[label] for label in y_part])
                X_list.append(X_part)
                y_list.append(y_part)
                groups_list.append(np.full(len(y_part), i))
            
            # Concatenate data across all participants
            X = np.concatenate(X_list, axis=0)
            y = np.concatenate(y_list, axis=0)
            groups = np.concatenate(groups_list, axis=0)
            
            # Create a pipeline: scaling is performed inside each CV fold to avoid leakage
            pipe = make_pipeline(StandardScaler(), model)

            outer_scores = []
            outer_best_params = []

            # Outer CV loop: leave-one-group-out
            for train_index, test_index in logo.split(X, y, groups):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]
                groups_train = groups[train_index]

                # Inner CV for hyperparameter tuning using GroupKFold
                inner_cv = GroupKFold(n_splits=5)
                rs = GridSearchCV(
                    estimator=pipe,
                    param_grid=param_grid,
                    cv=inner_cv,
                    scoring='accuracy',
                    n_jobs=n_jobs,
                    verbose=0
                )
                # Pass the groups for inner CV
                rs.fit(X_train, y_train, groups=groups_train)
                # Evaluate on the outer test fold
                test_score = 100 * rs.best_estimator_.score(X_test, y_test)
                outer_scores.append(test_score)
                outer_best_params.append(rs.best_params_)

            # Save results for the current mode
            model_score[mode] = {
                'logo_scores': outer_scores,
                'mean_score': np.mean(outer_scores),
                'fold_best_params': outer_best_params
            }

        # Save model scores to disk
        with open(f'processed_data/models/con_across_scores/{model_name}_scores.json', 'w') as f:
            json.dump(model_score, f)

#### CNN Model

In [ ]:
modes = ['face_type']

if run_dl_con_across_decoding:

    logo = LeaveOneGroupOut()

    model_score = {mode: None for mode in modes}
    for mode in modes:
        X_list, y_list, groups_list = [], [], []
        for i in range(ind_con[modes[0]][conditions_list[modes[0]][0]].shape[0]):
            participant_conds = []
            labels = []
            for condition in conditions_list[mode]:
                x = ind_con[mode][condition][i, dict_channel_types[channel_type], :, :, 0]
                label = np.array([condition] * x.shape[0])
                participant_conds.append(x)
                labels.append(label)

            # Stack the epochs and labels for this participant
            X_part = np.vstack(participant_conds)
            y_part = np.hstack(labels)

            # Standardize the data
            X_part = StandardScaler().fit_transform(X_part)
            
            # Map the labels to integers consistently
            label_mapping = {label: idx for idx, label in enumerate(np.unique(y_part))}
            y_part = np.array([label_mapping[label] for label in y_part])
            X_list.append(X_part)
            y_list.append(y_part)
            groups_list.append(np.full(len(y_part), i))
        
        # Concatenate data across all participants
        X = np.concatenate(X_list, axis=0)
        y = np.concatenate(y_list, axis=0)
        groups = np.concatenate(groups_list, axis=0)
        
        # Reshape the data to (n_samples, sqrt(n_features), sqrt(n_features))
        # This assumes that X.shape[1] is a perfect square.
        side_length = int(np.sqrt(X.shape[1]))
        X = X.reshape(X.shape[0], side_length, side_length)

        # Perform Leave-One-Group-Out CV; for each fold, reinitialize and train a new CNN model.
        fold_scores = []
        for train_idx, test_idx in logo.split(X, y, groups=groups):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            # Define a CNN model for the current fold
            model = tf.keras.Sequential([
                tf.keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2])),
                tf.keras.layers.Flatten(),

                tf.keras.layers.Dense(1024),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Activation('relu'),
                tf.keras.layers.Dropout(0.5),

                tf.keras.layers.Dense(512),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Activation('relu'),
                tf.keras.layers.Dropout(0.5),

                tf.keras.layers.Dense(128),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Activation('relu'),
                tf.keras.layers.Dropout(0.5),

                tf.keras.layers.Dense(64),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Activation('relu'),
                tf.keras.layers.Dropout(0.5),

                tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
            ])

            model.compile(
                optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
            )

            # Train the model using a validation split for early stopping.
            history = model.fit(
                X_train,
                y_train,
                epochs=100,
                batch_size=8,
                validation_split=0.2,
                callbacks=[
                    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
                    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5)
                ],
                verbose=1
            )

            # Evaluate on the test fold
            score = model.evaluate(X_test, y_test, verbose=0)
            plot_history(history, mode, score[1])
            break
    #         fold_scores.append(100 * score[1])  # score[1] is the accuracy

    #     # Compute the mean score for this mode
    #     model_score[mode] = {
    #         'logo_scores': fold_scores,
    #         'mean_score': np.mean(fold_scores)
    #     }

    # # Save the model scores to disk
    # with open(f'processed_data/models/con_across_scores/{model.__class__.__name__}_scores.json', 'w') as f:
    #     json.dump(model_score, f)

## Decoding Analysis Plots

### Accuracy Tables

In [ ]:
score_types = ['raw_within_scores', 'con_within_scores', 'raw_across_scores', 'con_across_scores']

if get_decoding_table_scores_plots:
    for score_type in score_types:
        # load data from 'processed_data/models' folder
        model_scores = {}
        # for any csv file in the folder
        for file in os.listdir(f'processed_data/models/{score_type}'):
            # if the file is a json file
            if file.endswith('.json'):
                # open the file
                with open(f'processed_data/models/{score_type}/{file}', 'r') as f:
                    # load the data from the file
                    model_scores[file.split('_')[0]] = json.load(f)

        # split score_type by '_'
        score_type_split = score_type.split('_')
        if score_type_split[0] == 'con':
            score_type_split[0] = 'Connectivity'
        elif score_type_split[0] == 'raw':
            score_type_split[0] = 'Raw'

        score_type_split[1] = score_type_split[1].capitalize()

        # Create a DataFrame from the model scores
        model_scores_df = pd.DataFrame([
            {'Model': model_name, 'Mode': mode, 'Mean Score': score['mean_score']}
            for model_name, scores in model_scores.items()
            for mode, score in scores.items()
        ])

        # Pivot the DataFrame to make 'Model' the first column and each 'Mode' a separate column
        model_scores_df = model_scores_df.pivot(index='Model', columns='Mode', values='Mean Score').reset_index()

        # Fill NaN values with 0 or any other value if needed
        model_scores_df.fillna(np.nan, inplace=True)

        # Remove the Mode column name
        model_scores_df.columns.name = None

        # Reorder the columns to have 'Model', 'all', 'face_type', 'emotion'
        model_scores_df = model_scores_df[['Model', 'all', 'face_type', 'emotion']]

        # Round the scores to 2 decimal places
        model_scores_df = model_scores_df.round(2)

        # Create a matplotlib figure and add a table
        fig, ax = plt.subplots(figsize=(6, len(model_scores_df) / 4))
        ax.axis('tight')
        ax.axis('off')
        table = ax.table(cellText=model_scores_df.values, colLabels=model_scores_df.columns, loc='center')

        # Set the biggest number in each column to be bold
        for i, col in enumerate(model_scores_df.columns):
            if col == 'Model':
                continue
            max_val = model_scores_df[col].max()
            for j, val in enumerate(model_scores_df[col]):
                if val == max_val:
                    table[(j + 1, i)].set_text_props(weight='bold')

        # Add a title to the plot
        ax.set_title(f'{score_type_split[0]} {score_type_split[1]} Scores (Accuracy%)')

        # Save the figure as a PNG file, bbox_inches='tight'
        plt.savefig(f'plots/models/tables/{score_type}.png', dpi=dpi, bbox_inches='tight')
        plt.close()

### Participant/LOGO Scores Plot

In [ ]:
if get_decoding_individual_scores_plots:
    for score_type in score_types:
        # load data from 'processed_data/models' folder
        model_scores = {}
        # for any csv file in the folder
        for file in os.listdir(f'processed_data/models/{score_type}'):
            # if the file is a json file
            if file.endswith('.json'):
                # open the file
                with open(f'processed_data/models/{score_type}/{file}', 'r') as f:
                    # load the data from the file
                    model_scores[file.split('_')[0]] = json.load(f)

        # split score_type by '_'
        score_type_split = score_type.split('_')
        if score_type_split[0] == 'con':
            score_type_split[0] = 'Connectivity'
        elif score_type_split[0] == 'raw':
            score_type_split[0] = 'Raw'

        score_type_split[1] = score_type_split[1].capitalize()

        models = list(model_scores.keys())
        modes = list(model_scores[list(model_scores.keys())[0]].keys())
        
        fig, ax = plt.subplots(figsize=(10, 6))
        colors = plt.cm.tab10.colors  # Use a colormap for distinct colors

        # Assign a consistent color to each model
        model_colors = {model: colors[idx % len(colors)] for idx, model in enumerate(models)}

        for mode_idx, mode in enumerate(modes):
            for model_idx, model in enumerate(models):
                scores = list(model_scores[model][mode].keys())[0]
                vp = plt.violinplot(
                    model_scores[model][mode][scores],
                    positions=[model_idx + mode_idx * (len(models) + 1)],
                    showmeans=True,
                    showextrema=True,
                    widths=0.8,
                    bw_method=0.5,
                )
                # Set the color for the violin parts
                for part in vp['bodies']:
                    part.set_facecolor(model_colors[model])
                    part.set_edgecolor('black')
                    part.set_alpha(0.7)
                vp['cmeans'].set_color('black')  # Median line color
                vp['cbars'].set_color('black')    # Whisker line color
                vp['cmaxes'].set_color('black')   # Max line color
                vp['cmins'].set_color('black')    # Min line color

        # Set x-ticks and labels
        x_ticks = [(len(models) + 1) * i + len(models) / 2 - 0.5 for i in range(len(modes))]
        ax.set_xticks(x_ticks)
        ax.set_xticklabels(modes)
        ax.set_xlabel('Modes')
        ax.set_ylabel('Scores')
        ax.set_title(f'{score_type_split[0]} {score_type_split[1]} Scores (Accuracy%)')
        ax.set_ylim(0, 100)
        ax.set_yticks(np.arange(0, 101, 10))
        ax.grid(axis='y', linestyle='--', alpha=0.7)
        ax.legend(
            handles=[mpatches.Patch(color=model_colors[model], label=model) for model in models],
            title='Models',
            loc='upper right'
        )
        plt.tight_layout()
        plt.savefig(f'plots/models/violin_plots/{score_type}.png', dpi=dpi / 2)
        plt.close()